In [ ]:
#CARICAMENTO LIBRERIE
import pandas as pd
import pymongo
import os
import datetime as dt
from kafka import KafkaProducer
from kafka import KafkaConsumer
import json
import time
import random
from pymongo import MongoClient
import simplejson
from twython import Twython
from bson.json_util import dumps
from bson import json_util
from random import randint
import re
import ekphrasis
from ekphrasis.classes.segmenter import Segmenter
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from datetime import timedelta
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
from collections import Counter
import pandas as pd
import nltk

import collections

# SET UP INIZIALE #

In [ ]:
#MI collego al client mongo e mi associo al Database Progetto:
client = MongoClient("mongo")
db = client.Progetto

In [ ]:
#Recupero la collection di tweets 1 di Gabri:
tysonjonesgabri = db.tysonjones_
tysonjonesgabri

Collection(Database(MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True), 'Progetto'), 'tysonjones_')

In [ ]:
#Recupero la collection di tweets:
tysonjonesteo = db.tysonjones
tysonjonesteo

Collection(Database(MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True), 'Progetto'), 'tysonjones')

In [ ]:
#Vado a caricare i risultati dell'incontro con i punteggi dei cartellini per l'incontro Tyson vs Jones
tabellino_incontro = pd.read_csv("/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/tabellino_incontro.csv", sep = ';')
tabellino_incontro = tabellino_incontro.drop(['Column 10','Column 11', 'Column 12', 'Column 13', 'Column 14'], axis=1)
tabellino_incontro_t = tabellino_incontro.transpose()
tabellino_incontro_t

,0,1
Column 1,Tyson,Jones
Column 2,10,9
Column 3,10,9
Column 4,9,10
Column 5,10,9
Column 6,10,9
Column 7,9,10
Column 8,9,10
Column 9,9,10


In [ ]:
new_header = tabellino_incontro_t.iloc[0] #grab the first row for the header
tabellino_incontro_t = tabellino_incontro_t[1:] #take the data less the header row
tabellino_incontro_t.columns = new_header #set the header row as the df header

In [ ]:
tabellino_incontro_t = tabellino_incontro_t.reset_index(drop = True)
tabellino_incontro_t = tabellino_incontro_t[["Tyson", "Jones"]]
tabellino_incontro_t['ROUND'] = range(1, 1 +len(tabellino_incontro_t))
tabellino_incontro_t = tabellino_incontro_t.rename(columns={"Tyson": "TYSON_punteggio_sing", "Jones": "JONES_punteggio_sing"})
tabellino_incontro_t

Column 1,TYSON_punteggio_sing,JONES_punteggio_sing,ROUND
0,10,9,1
1,10,9,2
2,9,10,3
3,10,9,4
4,10,9,5
5,9,10,6
6,9,10,7
7,9,10,8


In [ ]:
tabellino_incontro_t["TYSON_punteggio_sing"] = tabellino_incontro_t["TYSON_punteggio_sing"].astype(float)
tabellino_incontro_t["JONES_punteggio_sing"] = tabellino_incontro_t["JONES_punteggio_sing"].astype(float)

In [ ]:
tabellino_incontro_t['CUMSUM_punteggio_tyson'] = tabellino_incontro_t["TYSON_punteggio_sing"].cumsum()
tabellino_incontro_t['CUMSUM_punteggio_jones'] = tabellino_incontro_t["JONES_punteggio_sing"].cumsum()

In [ ]:
#Ottengo quindi il dataset con i risultati per quanto riguarda i tabellini, che sono il punteggio che i giudici assegnano. Il punteggio può andare da un minimo di 8 ad un massimo di 10.
tabellino_incontro_t

Column 1,TYSON_punteggio_sing,JONES_punteggio_sing,ROUND,CUMSUM_punteggio_tyson,CUMSUM_punteggio_jones
0,10.0,9.0,1,10.0,9.0
1,10.0,9.0,2,20.0,18.0
2,9.0,10.0,3,29.0,28.0
3,10.0,9.0,4,39.0,37.0
4,10.0,9.0,5,49.0,46.0
5,9.0,10.0,6,58.0,56.0
6,9.0,10.0,7,67.0,66.0
7,9.0,10.0,8,76.0,76.0


In [ ]:
tabellino_risultato_giudici = tabellino_incontro_t[tabellino_incontro_t["CUMSUM_punteggio_jones"] == 76.0]
tabellino_risultato_giudici = tabellino_risultato_giudici.reset_index()
tabellino_risultato_giudici

Column 1,index,TYSON_punteggio_sing,JONES_punteggio_sing,ROUND,CUMSUM_punteggio_tyson,CUMSUM_punteggio_jones
0,7,9.0,10.0,8,76.0,76.0


In [ ]:
#Vado a creare una lista da entrambe le collection, per poi trasformarle in DataFrame per una più semplificata maneggevolezza:
data_gabri = list(db.tysonjones_.find())
data_teo = list(db.tysonjones.find())

In [ ]:
#Traformo in Dataframe la collection dei Tweets di Gabri:
data_gabri = pd.DataFrame(data_gabri)
print("Nel dataset uno abbiamo: " + str(len(data_gabri)) + " tweets")

#Traformo in Dataframe la collection dei Tweets di Teo:
data_teo = pd.DataFrame(data_teo)
print("Nel dataset due abbiamo: " + str(len(data_teo)) + " tweets")

Nel dataset uno abbiamo: 508225 tweets
Nel dataset due abbiamo: 259248 tweets


In [ ]:
#Aggiungo un'ora al dataset raccolto da Gabri
data_gabri['Date_time'] = pd.to_datetime(data_gabri['Date_time'])
data_gabri['Date_time'] = pd.DatetimeIndex(data_gabri['Date_time']) + timedelta(hours=1)

In [ ]:
#Effetto l'unione delle due collections di tweest in un unico dataframe:
frames_dataframe = [data_teo, data_gabri]
dataset_tweets_complessivo = pd.concat(frames_dataframe)
print("Nel dataset complessivo abbiamo a disposizione: " + str(len(dataset_tweets_complessivo)) + " tweets")

Nel dataset complessivo abbiamo a disposizione: 767473 tweets


In [ ]:
#ESTRAPOLO GLI HASHTAG DAL TESTO E CREO UNA COLONNA APPOSITA 
dataset_tweets_complessivo["hashtags"] = dataset_tweets_complessivo["Text"].apply(lambda x: re.findall(r"#(\w+)", x))

In [ ]:
dataset_tweets_complessivo = dataset_tweets_complessivo.rename(columns={'Date_time':'Date_time_italy'})

#ELIMINAZIONE BOT 
Si contano il numero di volte che un utente twetta, utenti con un numero elevato di tweets, pochi followers e testi relativi a pubblicità e ripetitivi vengono eliminati. 

In [ ]:
from collections import Counter

In [ ]:
 see = dataset_tweets_complessivo.Screen_name.value_counts().reset_index(name='Count').rename(columns = {'index':'A'})
 see.head(20)

,A,Count
0,PugilistSteve,525
1,labony20942855,112
2,UFCFightsStream,111
3,triller,110
4,idnpublic,110
5,idnvideo,108
6,gabriel_gabdiel,105
7,THETOPICisCOOL,105
8,moe_shaq_24,105
9,lopezsij207,104


In [ ]:
rslt_df = dataset_tweets_complessivo[dataset_tweets_complessivo['Screen_name']=='SKYTVLiveStrea1']

In [ ]:
rslt_df

,_id,Screen_name,Name,Text,Followers,Number_Retweet,Position,Date_time,Hashtags,Date_time_italy,count
35546,{'$oid': '5fc207436b90cd1acfacf5ac'},SKYTVLiveStrea1,SKY TV Live Stream,Watch New Zealand vs Argentina Live Streaming ...,3,0,None,2020-11-28 08:15:58,[],NaT,bull97rock
35883,{'$oid': '5fc209206b90cd1acfacf6fd'},SKYTVLiveStrea1,SKY TV Live Stream,Watch New Zealand vs Argentina Live Streaming ...,3,0,None,2020-11-28 08:23:55,[],NaT,__KagedAnimal
36471,{'$oid': '5fc20cea6b90cd1acfacf949'},SKYTVLiveStrea1,SKY TV Live Stream,Watch New Zealand vs Argentina Live Streaming ...,3,0,None,2020-11-28 08:40:05,[],NaT,Champagne_Jefe
36652,{'$oid': '5fc20df06b90cd1acfacf9fe'},SKYTVLiveStrea1,SKY TV Live Stream,Watch New Zealand vs Argentina Live Streaming ...,3,0,None,2020-11-28 08:44:26,[],NaT,CaerwynnKani
53954,{'$oid': '5fc26b976b90cd1acfad3d94'},SKYTVLiveStrea1,SKY TV Live Stream,Watch Mike Tyson vs Roy Jones Jr Live Streamin...,3,0,None,2020-11-28 15:24:02,[],NaT,_StendoBans__
...,...,...,...,...,...,...,...,...,...,...,...
286138,{'$oid': '5fc32bd3fec2296bd50b618e'},SKYTVLiveStrea1,SKY TV Live Stream,Watch Mike Tyson vs Roy Jones Jr Live Streamin...,3,0,None,2020-11-29 05:04:13,[],2020-11-29 06:04:13,ttazz77
294937,{'$oid': '5fc32c84fec2296bd50b83ed'},SKYTVLiveStrea1,SKY TV Live Stream,Now Happened!!\nYou can see Live 4K now!!\n\nW...,3,0,None,2020-11-29 05:07:11,[],2020-11-29 06:07:11,yofavlightskn
296449,{'$oid': '5fc32ca4fec2296bd50b89d5'},SKYTVLiveStrea1,SKY TV Live Stream,Watch Mike Tyson vs Roy Jones Jr Live Streamin...,3,0,None,2020-11-29 05:07:43,[],2020-11-29 06:07:43,crocodilius
330345,{'$oid': '5fc32f55fec2296bd50c0e3d'},SKYTVLiveStrea1,SKY TV Live Stream,Watch Mike Tyson vs Roy Jones Jr Live Streamin...,3,0,None,2020-11-29 05:19:12,[],2020-11-29 06:19:12,NBAquickreport


In [ ]:
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'LauraChimanski'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'KasimAkaNinja'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'allsportlive365'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == '	LeannThoman1455'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == '_PequenaDoLubs'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'DarckoGV'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'CasaBourbon'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'caseys508'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'EllysonLyma'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'gulruhati'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'Wahyu_Handika'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'KasimAkaNinja'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'obby_2013'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MatchRd'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == '	MahadiHasanAsh2'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'billah_vai	'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'AusOpenAustral1'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MikeTys98721109'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'billah_vai'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MahadiHasanAsh2'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'Kfernandiz'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'LeannThoman1455'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'uefapreview'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MikeTys10'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'ZENAPRCHAMP'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'FiteTV'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'DimeDroppingNFL'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'natalia_12love'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'Jay_Dekassia'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'Robel73401463'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'SeLca_official'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'LeannThoman1455'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MahadiHasanAsh2'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'UFC254Khabibvs1'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MonicaBegay13'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'Miranda17752622'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'AngieWo44433920'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'AllEveantLive'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'JamesStrafela'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'tsunamishon'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'FestivalLive4'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'boxinghd4'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'boxingst2'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'SKYTVLiveStrea1'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MDShari48779970	'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'sharif29801314'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'suktara05745494	'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'streams2019'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'GegumMst'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'SunSport'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'mstsuraiya2	'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'mstparvin6'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'SportsEvent9'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'GarzonMel'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'SPORTNE97628454'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MikeTys9'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'AmyBake04962735'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'TrinaSa91580331'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'CESBoxing'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'ismail66480207	'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'mmastreamslive'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'CouchcastSports'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'NaomiOs23081163'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'alina48148880'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'MDShari48779970'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'suktara05745494'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'mstsuraiya2'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'ismail66480207'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'dw504966'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'lewisgfe437'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'clarkgsu340'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'martingyc509'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'Stephan49216456'].index)
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(dataset_tweets_complessivo[dataset_tweets_complessivo.Screen_name == 'hallbvv218'].index)

In [ ]:
#ESTRAPOLO GLI HASHTAG DAL TESTO E CREO UNA COLONNA APPOSITA 
dataset_tweets_complessivo["hashtags"] = dataset_tweets_complessivo["Text"].apply(lambda x: re.findall(r"#(\w+)", x))

In [ ]:
#PULIZIA DEL DATASET ELIMINANDO LE COLONNE CHE NON SERVONO 
dataset_tweets_complessivo = dataset_tweets_complessivo.drop(['Screen_name','Name','Followers',	'Number_Retweet','Position','Hashtags'], axis=1)
dataset_tweets_complessivo = dataset_tweets_complessivo.rename(columns={'Date_time':'Date_time_italy'})

# Per la costruzione delle statistiche riferite al match, abbiamo estratto le informazioni tramite fonte

In [ ]:
#[incontro_powerpunches_landed]

In [ ]:
#Costruisco dataset legato ai pugni dati da Tyson e Jones:
incontro_powerpunches_landed = pd.read_csv("/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/statistics-.csv",  header= None)
incontro_powerpunches_landed = incontro_powerpunches_landed.transpose()

new_header_incontro_powerpunches_landed = incontro_powerpunches_landed.iloc[0] 
incontro_powerpunches_landed = incontro_powerpunches_landed[1:] 
incontro_powerpunches_landed.columns = new_header_incontro_powerpunches_landed

incontro_powerpunches_landed = incontro_powerpunches_landed.reset_index(drop = True)
incontro_powerpunches_landed

,TYSON,Power Punches landed / Thrown Tyson,JONES,Power Punches landed / Thrown Jones,ROUND
0,6/11,0.545,0/2,0,1
1,8/16,0.500,2/19,0.105,2
2,8/21,0.381,6/24,0.250,3
3,4115,0.267,3/16,0.188,4
4,9/20,0.450,6/29,0.207,5
5,5/12,0.417,5/18,0.278,6
6,6/17,0.353,2/19,0.105,7
7,11/25,0.440,4/24,0.167,8


In [ ]:
incontro_powerpunches_landed.loc[incontro_powerpunches_landed.TYSON == 4115, 'TYSON'] = "4/15"

In [ ]:
incontro_powerpunches_landed['TYSON'] = incontro_powerpunches_landed['TYSON'].astype(str)
incontro_powerpunches_landed['JONES'] = incontro_powerpunches_landed['JONES'].astype(str)

In [ ]:
incontro_powerpunches_landed["TYSON"] = incontro_powerpunches_landed["TYSON"].str.replace(' ', '')

incontro_powerpunches_landed["power_pugni_tirati_tyson"] = incontro_powerpunches_landed["TYSON"].apply(lambda x: re.findall(r"/(\w+)", x))
incontro_powerpunches_landed["power_pugni_tirati_jones"] = incontro_powerpunches_landed["JONES"].apply(lambda x: re.findall(r"/(\w+)", x))

incontro_powerpunches_landed["tot_punch_tirati_tyson"] = incontro_powerpunches_landed["TYSON"].apply(lambda x: re.findall(r"(\w+)/", x))
incontro_powerpunches_landed["tot_punch_tirati_jones"] = incontro_powerpunches_landed["JONES"].apply(lambda x: re.findall(r"(\w+)/", x))

In [ ]:
incontro_powerpunches_landed = incontro_powerpunches_landed.explode("power_pugni_tirati_tyson")
incontro_powerpunches_landed = incontro_powerpunches_landed.explode("power_pugni_tirati_jones")
incontro_powerpunches_landed = incontro_powerpunches_landed.explode("tot_punch_tirati_tyson")
incontro_powerpunches_landed = incontro_powerpunches_landed.explode("tot_punch_tirati_jones")
incontro_powerpunches_landed

,TYSON,Power Punches landed / Thrown Tyson,JONES,Power Punches landed / Thrown Jones,ROUND,power_pugni_tirati_tyson,power_pugni_tirati_jones,tot_punch_tirati_tyson,tot_punch_tirati_jones
0,6/11,0.545,0/2,0,1,11,2,6,0
1,8/16,0.500,2/19,0.105,2,16,19,8,2
2,8/21,0.381,6/24,0.250,3,21,24,8,6
3,4/15,0.267,3/16,0.188,4,15,16,4,3
4,9/20,0.450,6/29,0.207,5,20,29,9,6
5,5/12,0.417,5/18,0.278,6,12,18,5,5
6,6/17,0.353,2/19,0.105,7,17,19,6,2
7,11/25,0.440,4/24,0.167,8,25,24,11,4


In [ ]:
incontro_powerpunches_landed["power_pugni_tirati_tyson"] = incontro_powerpunches_landed["power_pugni_tirati_tyson"].astype(float)
incontro_powerpunches_landed["power_pugni_tirati_jones"] = incontro_powerpunches_landed["power_pugni_tirati_jones"].astype(float)
incontro_powerpunches_landed["tot_punch_tirati_tyson"] = incontro_powerpunches_landed["tot_punch_tirati_tyson"].astype(float)
incontro_powerpunches_landed["tot_punch_tirati_jones"] = incontro_powerpunches_landed["tot_punch_tirati_jones"].astype(float)
incontro_powerpunches_landed["Power Punches landed / Thrown Tyson"] = incontro_powerpunches_landed["Power Punches landed / Thrown Tyson"].astype(float)
incontro_powerpunches_landed["Power Punches landed / Thrown Jones"] = incontro_powerpunches_landed["Power Punches landed / Thrown Jones"].astype(float)

In [ ]:
incontro_powerpunches_landed

,TYSON,Power Punches landed / Thrown Tyson,JONES,Power Punches landed / Thrown Jones,ROUND,power_pugni_tirati_tyson,power_pugni_tirati_jones,tot_punch_tirati_tyson,tot_punch_tirati_jones
0,6/11,0.545,0/2,0.000,1,11.0,2.0,6.0,0.0
1,8/16,0.500,2/19,0.105,2,16.0,19.0,8.0,2.0
2,8/21,0.381,6/24,0.250,3,21.0,24.0,8.0,6.0
3,4/15,0.267,3/16,0.188,4,15.0,16.0,4.0,3.0
4,9/20,0.450,6/29,0.207,5,20.0,29.0,9.0,6.0
5,5/12,0.417,5/18,0.278,6,12.0,18.0,5.0,5.0
6,6/17,0.353,2/19,0.105,7,17.0,19.0,6.0,2.0
7,11/25,0.440,4/24,0.167,8,25.0,24.0,11.0,4.0


In [ ]:
incontro_powerpunches_landed['CUMSUM_power_tirati_tyson'] = incontro_powerpunches_landed["power_pugni_tirati_tyson"].cumsum()
incontro_powerpunches_landed['CUMSUM_power_tirati_jones'] = incontro_powerpunches_landed["power_pugni_tirati_jones"].cumsum()
incontro_powerpunches_landed['CUMSUM_pugni_2_landed_tyson'] = incontro_powerpunches_landed["tot_punch_tirati_tyson"].cumsum()
incontro_powerpunches_landed['CUMSUM_pugni_2_landed_jones'] = incontro_powerpunches_landed["tot_punch_tirati_jones"].cumsum()
incontro_powerpunches_landed

,TYSON,Power Punches landed / Thrown Tyson,JONES,Power Punches landed / Thrown Jones,ROUND,power_pugni_tirati_tyson,power_pugni_tirati_jones,tot_punch_tirati_tyson,tot_punch_tirati_jones,CUMSUM_power_tirati_tyson,CUMSUM_power_tirati_jones,CUMSUM_pugni_2_landed_tyson,CUMSUM_pugni_2_landed_jones
0,6/11,0.545,0/2,0.000,1,11.0,2.0,6.0,0.0,11.0,2.0,6.0,0.0
1,8/16,0.500,2/19,0.105,2,16.0,19.0,8.0,2.0,27.0,21.0,14.0,2.0
2,8/21,0.381,6/24,0.250,3,21.0,24.0,8.0,6.0,48.0,45.0,22.0,8.0
3,4/15,0.267,3/16,0.188,4,15.0,16.0,4.0,3.0,63.0,61.0,26.0,11.0
4,9/20,0.450,6/29,0.207,5,20.0,29.0,9.0,6.0,83.0,90.0,35.0,17.0
5,5/12,0.417,5/18,0.278,6,12.0,18.0,5.0,5.0,95.0,108.0,40.0,22.0
6,6/17,0.353,2/19,0.105,7,17.0,19.0,6.0,2.0,112.0,127.0,46.0,24.0
7,11/25,0.440,4/24,0.167,8,25.0,24.0,11.0,4.0,137.0,151.0,57.0,28.0


In [ ]:
incontro_powerpunches_landed = incontro_powerpunches_landed.rename(columns={"TYSON": "TYSON_power_total", "JONES": "JONES_power_total"})
incontro_powerpunches_landed.head(1)

,TYSON_power_total,Power Punches landed / Thrown Tyson,JONES_power_total,Power Punches landed / Thrown Jones,ROUND,power_pugni_tirati_tyson,power_pugni_tirati_jones,tot_punch_tirati_tyson,tot_punch_tirati_jones,CUMSUM_power_tirati_tyson,CUMSUM_power_tirati_jones,CUMSUM_pugni_2_landed_tyson,CUMSUM_pugni_2_landed_jones
0,6/11,0.545,0/2,0.0,1,11.0,2.0,6.0,0.0,11.0,2.0,6.0,0.0


In [ ]:
#[incontro_punches_total]

In [ ]:
#Costruisco dataset legato ai pugni dati da Tyson e Jones:
incontro_punches_total = pd.read_csv("/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/statistics-1.csv",  header= None)
incontro_punches_total = incontro_punches_total.transpose()

new_header_incontro_punches_total = incontro_punches_total.iloc[0] 
incontro_punches_total = incontro_punches_total[1:] 
incontro_punches_total.columns = new_header_incontro_punches_total

incontro_punches_total = incontro_punches_total.reset_index(drop = True)
incontro_punches_total

,TYSON,Total Punches Landed / Thrown Tyson,JONES,Total Punches Landed / Thrown Jones,ROUND
0,7/ 22,0.318,2/18,0.111,1
1,8/21,0.381,4/32,0.125,2
2,9/28,0.321,6/30,0.200,3
3,6/21,0.286,3128,0.107,4
4,12130,0.400,9/42,0.214,5
5,6/19,0.316,6/28,0.214,6
6,8/23,0.348,2/25,0.080,7
7,11/29,0.379,5/33,0.152,8


In [ ]:
incontro_punches_total.loc[incontro_punches_total.TYSON == 12130, 'TYSON'] = "12/30"
incontro_punches_total.loc[incontro_punches_total.JONES == 3128, 'JONES'] = "3/28"

In [ ]:
incontro_punches_total['TYSON'] = incontro_punches_total['TYSON'].astype(str)
incontro_punches_total['JONES'] = incontro_punches_total['JONES'].astype(str)

In [ ]:
incontro_punches_total["TYSON"] = incontro_punches_total["TYSON"].str.replace(' ', '')

incontro_punches_total["pugni_tirati_complex_tyson"] = incontro_punches_total["TYSON"].apply(lambda x: re.findall(r"/(\w+)", x))
incontro_punches_total["pugni_tirati_complex_jones"] = incontro_punches_total["JONES"].apply(lambda x: re.findall(r"/(\w+)", x))

incontro_punches_total["tot_punch_landed_tyson"] = incontro_punches_total["TYSON"].apply(lambda x: re.findall(r"(\w+)/", x))
incontro_punches_total["tot_punch_landed_jones"] = incontro_punches_total["JONES"].apply(lambda x: re.findall(r"(\w+)/", x))

In [ ]:
incontro_punches_total = incontro_punches_total.explode("pugni_tirati_complex_tyson")
incontro_punches_total = incontro_punches_total.explode("pugni_tirati_complex_jones")
incontro_punches_total = incontro_punches_total.explode("tot_punch_landed_tyson")
incontro_punches_total = incontro_punches_total.explode("tot_punch_landed_jones")
incontro_punches_total

,TYSON,Total Punches Landed / Thrown Tyson,JONES,Total Punches Landed / Thrown Jones,ROUND,pugni_tirati_complex_tyson,pugni_tirati_complex_jones,tot_punch_landed_tyson,tot_punch_landed_jones
0,7/22,0.318,2/18,0.111,1,22,18,7,2
1,8/21,0.381,4/32,0.125,2,21,32,8,4
2,9/28,0.321,6/30,0.200,3,28,30,9,6
3,6/21,0.286,3/28,0.107,4,21,28,6,3
4,12/30,0.400,9/42,0.214,5,30,42,12,9
5,6/19,0.316,6/28,0.214,6,19,28,6,6
6,8/23,0.348,2/25,0.080,7,23,25,8,2
7,11/29,0.379,5/33,0.152,8,29,33,11,5


In [ ]:
incontro_punches_total["pugni_tirati_complex_tyson"] = incontro_punches_total["pugni_tirati_complex_tyson"].astype(float)
incontro_punches_total["pugni_tirati_complex_jones"] = incontro_punches_total["pugni_tirati_complex_jones"].astype(float)
incontro_punches_total["tot_punch_landed_tyson"] = incontro_punches_total["tot_punch_landed_tyson"].astype(float)
incontro_punches_total["tot_punch_landed_jones"] = incontro_punches_total["tot_punch_landed_jones"].astype(float)
incontro_punches_total["Total Punches Landed / Thrown Tyson"] = incontro_punches_total["Total Punches Landed / Thrown Tyson"].astype(float)
incontro_punches_total["Total Punches Landed / Thrown Jones"] = incontro_punches_total["Total Punches Landed / Thrown Jones"].astype(float)

In [ ]:
incontro_punches_total['CUMSUM_pugni_tirati_tyson'] = incontro_punches_total["pugni_tirati_complex_tyson"].cumsum()
incontro_punches_total['CUMSUM_pugni_tirati_jones'] = incontro_punches_total["pugni_tirati_complex_jones"].cumsum()
incontro_punches_total['CUMSUM_pugni_landed_tyson'] = incontro_punches_total["tot_punch_landed_tyson"].cumsum()
incontro_punches_total['CUMSUM_pugni_landed_jones'] = incontro_punches_total["tot_punch_landed_jones"].cumsum()

In [ ]:
incontro_punches_total

,TYSON,Total Punches Landed / Thrown Tyson,JONES,Total Punches Landed / Thrown Jones,ROUND,pugni_tirati_complex_tyson,pugni_tirati_complex_jones,tot_punch_landed_tyson,tot_punch_landed_jones,CUMSUM_pugni_tirati_tyson,CUMSUM_pugni_tirati_jones,CUMSUM_pugni_landed_tyson,CUMSUM_pugni_landed_jones
0,7/22,0.318,2/18,0.111,1,22.0,18.0,7.0,2.0,22.0,18.0,7.0,2.0
1,8/21,0.381,4/32,0.125,2,21.0,32.0,8.0,4.0,43.0,50.0,15.0,6.0
2,9/28,0.321,6/30,0.200,3,28.0,30.0,9.0,6.0,71.0,80.0,24.0,12.0
3,6/21,0.286,3/28,0.107,4,21.0,28.0,6.0,3.0,92.0,108.0,30.0,15.0
4,12/30,0.400,9/42,0.214,5,30.0,42.0,12.0,9.0,122.0,150.0,42.0,24.0
5,6/19,0.316,6/28,0.214,6,19.0,28.0,6.0,6.0,141.0,178.0,48.0,30.0
6,8/23,0.348,2/25,0.080,7,23.0,25.0,8.0,2.0,164.0,203.0,56.0,32.0
7,11/29,0.379,5/33,0.152,8,29.0,33.0,11.0,5.0,193.0,236.0,67.0,37.0


In [ ]:
incontro_punches_total = incontro_punches_total.rename(columns={"TYSON": "TYSON_punches_total", "JONES": "JONES_punches_total"})
incontro_punches_total.head(1)

,TYSON_punches_total,Total Punches Landed / Thrown Tyson,JONES_punches_total,Total Punches Landed / Thrown Jones,ROUND,pugni_tirati_complex_tyson,pugni_tirati_complex_jones,tot_punch_landed_tyson,tot_punch_landed_jones,CUMSUM_pugni_tirati_tyson,CUMSUM_pugni_tirati_jones,CUMSUM_pugni_landed_tyson,CUMSUM_pugni_landed_jones
0,7/22,0.318,2/18,0.111,1,22.0,18.0,7.0,2.0,22.0,18.0,7.0,2.0


In [ ]:
#[INCONTRO_JAB_LANDED]:

In [ ]:
#Costruisco dataset legato ai pugni dati da Tyson e Jones:
incontro_jab_landed = pd.read_csv("/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/statistics-2.csv", header= None)
incontro_jab_landed = incontro_jab_landed.transpose()

new_header_incontro_jab_landed = incontro_jab_landed.iloc[0] 
incontro_jab_landed = incontro_jab_landed[1:] 
incontro_jab_landed.columns = new_header_incontro_jab_landed

incontro_jab_landed = incontro_jab_landed.reset_index(drop = True)
incontro_jab_landed

,TYSON,Jab Landed / Thrown Tyson,JONES,Jab Landed / Thrown Jones,ROUND
0,1/ 11,0.091,2/16,0.125,1
1,0/5,0,2/13,0.154,2
2,1/7,0.143,0/6,0,3
3,2/6,0.333,0/12,0,4
4,3/10,0.300,3/13,0.231,5
5,1/7,0.143,1/10,0.100,6
6,2/6,0.333,0/6,0,7
7,0/4,0,1/9,0.111,8


In [ ]:
incontro_jab_landed["TYSON"] = incontro_jab_landed["TYSON"].str.replace(' ', '')

incontro_jab_landed["pugni_tot_tyson"] = incontro_jab_landed["TYSON"].apply(lambda x: re.findall(r"/(\w+)", x))
incontro_jab_landed["pugni_tot_jones"] = incontro_jab_landed["JONES"].apply(lambda x: re.findall(r"/(\w+)", x))

incontro_jab_landed["jab_landed_tyson"] = incontro_jab_landed["TYSON"].apply(lambda x: re.findall(r"(\w+)/", x))
incontro_jab_landed["jab_landed_jones"] = incontro_jab_landed["JONES"].apply(lambda x: re.findall(r"(\w+)/", x))

In [ ]:
incontro_jab_landed = incontro_jab_landed.explode("pugni_tot_tyson")
incontro_jab_landed = incontro_jab_landed.explode("pugni_tot_jones")
incontro_jab_landed = incontro_jab_landed.explode("jab_landed_tyson")
incontro_jab_landed = incontro_jab_landed.explode("jab_landed_jones")
incontro_jab_landed

,TYSON,Jab Landed / Thrown Tyson,JONES,Jab Landed / Thrown Jones,ROUND,pugni_tot_tyson,pugni_tot_jones,jab_landed_tyson,jab_landed_jones
0,1/11,0.091,2/16,0.125,1,11,16,1,2
1,0/5,0,2/13,0.154,2,5,13,0,2
2,1/7,0.143,0/6,0,3,7,6,1,0
3,2/6,0.333,0/12,0,4,6,12,2,0
4,3/10,0.300,3/13,0.231,5,10,13,3,3
5,1/7,0.143,1/10,0.100,6,7,10,1,1
6,2/6,0.333,0/6,0,7,6,6,2,0
7,0/4,0,1/9,0.111,8,4,9,0,1


In [ ]:
incontro_jab_landed["pugni_tot_tyson"] = incontro_jab_landed["pugni_tot_tyson"].astype(float)
incontro_jab_landed["pugni_tot_jones"] = incontro_jab_landed["pugni_tot_jones"].astype(float)
incontro_jab_landed["jab_landed_tyson"] = incontro_jab_landed["jab_landed_tyson"].astype(float)
incontro_jab_landed["jab_landed_jones"] = incontro_jab_landed["jab_landed_jones"].astype(float)
incontro_jab_landed["Jab Landed / Thrown Tyson"] = incontro_jab_landed["Jab Landed / Thrown Tyson"].astype(float)
incontro_jab_landed["Jab Landed / Thrown Jones"] = incontro_jab_landed["Jab Landed / Thrown Jones"].astype(float)

In [ ]:
incontro_jab_landed.dtypes

0
TYSON                         object
Jab Landed / Thrown Tyson    float64
JONES                         object
Jab Landed / Thrown Jones    float64
ROUND                         object
pugni_tot_tyson              float64
pugni_tot_jones              float64
jab_landed_tyson             float64
jab_landed_jones             float64
dtype: object

In [ ]:
incontro_jab_landed['CUMSUM_pugni_tyson'] = incontro_jab_landed['pugni_tot_tyson'].cumsum()
incontro_jab_landed['CUMSUM_pugni_jones'] = incontro_jab_landed['pugni_tot_jones'].cumsum()
incontro_jab_landed['CUMSUM_jab_landed_tyson'] = incontro_jab_landed['jab_landed_tyson'].cumsum()
incontro_jab_landed['CUMSUM_jab_landed_jones'] = incontro_jab_landed['jab_landed_jones'].cumsum()

In [ ]:
incontro_jab_landed

,TYSON,Jab Landed / Thrown Tyson,JONES,Jab Landed / Thrown Jones,ROUND,pugni_tot_tyson,pugni_tot_jones,jab_landed_tyson,jab_landed_jones,CUMSUM_pugni_tyson,CUMSUM_pugni_jones,CUMSUM_jab_landed_tyson,CUMSUM_jab_landed_jones
0,1/11,0.091,2/16,0.125,1,11.0,16.0,1.0,2.0,11.0,16.0,1.0,2.0
1,0/5,0.000,2/13,0.154,2,5.0,13.0,0.0,2.0,16.0,29.0,1.0,4.0
2,1/7,0.143,0/6,0.000,3,7.0,6.0,1.0,0.0,23.0,35.0,2.0,4.0
3,2/6,0.333,0/12,0.000,4,6.0,12.0,2.0,0.0,29.0,47.0,4.0,4.0
4,3/10,0.300,3/13,0.231,5,10.0,13.0,3.0,3.0,39.0,60.0,7.0,7.0
5,1/7,0.143,1/10,0.100,6,7.0,10.0,1.0,1.0,46.0,70.0,8.0,8.0
6,2/6,0.333,0/6,0.000,7,6.0,6.0,2.0,0.0,52.0,76.0,10.0,8.0
7,0/4,0.000,1/9,0.111,8,4.0,9.0,0.0,1.0,56.0,85.0,10.0,9.0


In [ ]:
incontro_jab_landed = incontro_jab_landed.rename(columns={"TYSON": "TYSON_jab_landed", "JONES": "JONES_jab_landed"})
incontro_jab_landed.head(1)

,TYSON_jab_landed,Jab Landed / Thrown Tyson,JONES_jab_landed,Jab Landed / Thrown Jones,ROUND,pugni_tot_tyson,pugni_tot_jones,jab_landed_tyson,jab_landed_jones,CUMSUM_pugni_tyson,CUMSUM_pugni_jones,CUMSUM_jab_landed_tyson,CUMSUM_jab_landed_jones
0,1/11,0.091,2/16,0.125,1,11.0,16.0,1.0,2.0,11.0,16.0,1.0,2.0


# PRE-PROCESSING TESTO TEXT_CLEAN_1 #

In [ ]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
    r"\U0001F600-\U0001F64F"  # emoticons
    r"\U0001F300-\U0001F5FF"  # symbols & pictographs
    r"\U0001F680-\U0001F6FF"  # transport & map symbols
    r"\U0001F1E0-\U0001F1FF"  #flags
    r"\U00002702-\U000027B0"
    r"\U000024C2-\U0001F251"# flags (iOS) 
    r"\U0001f926-\U0001f937"
    r"\U00010000-\U0010ffff"
    r"\u2640-\u2642"
    r"\u2600-\u2B55"
    r"\u200d"
    r"\u23cf"
    r"\u23e9"
    r"\u231a"
    r"\ufe0f"  # dingbats
    r"\u3030"
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

dataset_tweets_complessivo['Text_clean_1'] = dataset_tweets_complessivo['Text'].apply(preprocess)

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop =stopwords.words('english')
import string
punctuation = string.punctuation

def preprocess(tokenized_sentence):
  tokenized_sentence_stop = [item.lower() for item in tokenized_sentence if item.lower() not in stop] #remove stopwords and lowercase
  return [item for item in tokenized_sentence_stop if item not in punctuation] #remove punctuation 

dataset_tweets_complessivo['Text_clean_1'] = dataset_tweets_complessivo['Text_clean_1'].apply(preprocess)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matteo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from collections import Counter
import itertools

sentences = (list(itertools.chain(dataset_tweets_complessivo["Text_clean_1"])))
flat_list = [item for sublist in sentences for item in sublist]
flat_list[:30]

c = Counter(flat_list)
c.most_common(10)

[('rt', 372150),
 ('tyson', 358710),
 ('jones', 253808),
 ('mike', 220209),
 ('roy', 165119),
 ('fight', 124825),
 ('#tysonvsjones', 110503),
 ('jr', 100897),
 ('vs', 89224),
 ('live', 83864)]

In [ ]:
stop = set(stop)
stop.add("rt")
stop.add("…")
stop.add("n")
stop.add("’".lower())
stop.add("🇺")
stop.add("s")
stop.add("S".lower())
stop.add("nstay".lower())
stop.add(":/")
stop.add("nhttps".lower())
stop.add("https".lower())
stop.add("I")
stop.add("The")
stop.add("“")
stop.add("”")
stop.add("u")
stop.add("🇸")
stop.add("e")
stop.add("n")
stop.add("o")
stop.add("t")
stop.add("r")
stop.add("vs")


stop = list(stop)
print(stop)

['ain', 'ours', 'what', "shan't", "doesn't", 'll', 'been', "it's", 'our', 'those', 'who', 'having', "should've", '🇸', 't', 'should', 'wasn', 'with', 'rt', 'until', 'won', 'she', "weren't", '…', 'yourselves', 'so', 'themselves', 'where', 'between', 'about', "wouldn't", 'has', "hasn't", 'not', 'a', "won't", 'I', 'through', 'own', 'isn', 'are', 'now', 'to', 'at', 'while', "shouldn't", 'mustn', 'because', 'into', 'itself', "mustn't", 'd', 'mightn', 'needn', 'most', 'which', 'being', 'nstay', "you've", 'ma', 'your', 'all', 'me', 'am', "mightn't", 'nhttps', "you'd", "didn't", 'him', 'some', 'herself', 'under', 'their', 'above', 'on', 'can', "you'll", 'when', 'have', 'again', 'be', "wasn't", 'don', 'by', 'how', 'out', 'hadn', 'against', 'too', 'up', 'had', 'and', 'does', 'from', 'm', "needn't", 'n', 'this', 'myself', 'r', '“', 'whom', 'both', 'it', 'then', 'such', 'hers', 's', 'e', 'https', 'nor', 'they', 'any', ':/', 'down', 'he', "that'll", 'hasn', 'an', 'before', 'below', 'just', "aren't",

In [ ]:
def preprocess(tokenized_sentence, stopwords=[]):
  tokenized_sentence_stop = [item.lower() for item in tokenized_sentence if item.lower() not in set(stopwords)] #remove stopwords and lowercase
  return [item.lower() for item in tokenized_sentence_stop if item not in punctuation] #remove punctuation 


dataset_tweets_complessivo['Text_clean_1'] = dataset_tweets_complessivo['Text_clean_1'].apply(lambda x: preprocess(x, stopwords=stop)) 

In [ ]:
sentences = (list(itertools.chain(dataset_tweets_complessivo['Text_clean_1'])))
flat_list = [item for sublist in sentences for item in sublist]
flat_list[:30]

c = Counter(flat_list)
c.most_common(25)

[('tyson', 358710),
 ('jones', 253808),
 ('mike', 220209),
 ('roy', 165119),
 ('fight', 124825),
 ('#tysonvsjones', 110503),
 ('jr', 100897),
 ('live', 83864),
 ('#tysonjones', 75749),
 ('nate', 61672),
 ('tonight', 49292),
 ('robinson', 48449),
 ('like', 41184),
 ('paul', 40994),
 ('jake', 40079),
 ('stream', 39574),
 ('watch', 34883),
 ('@miketyson', 34275),
 ('boxing', 33551),
 ('got', 32892),
 ('fighting', 31702),
 ('snoop', 30884),
 ('draw', 30045),
 ('one', 25217),
 ('get', 22891)]

In [ ]:
#LEMMATIZZAZIONE 
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
nltk.download('wordnet')

def preprocess(tokenized_sentence, stopwords=[]):
  tokenized_sentence_stop = [item.lower() for item in tokenized_sentence if item.lower() not in set(stopwords)] #remove stopwords and lowercase
  x = [item.lower() for item in tokenized_sentence_stop if item not in punctuation] #remove punctuation 
  return [lemmatizer.lemmatize(item) for item in x]


dataset_tweets_complessivo["Text_clean_1"]= dataset_tweets_complessivo["Text_clean_1"].apply(lambda x: preprocess(x, stopwords=stop))

[nltk_data] Downloading package wordnet to /Users/matteo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
sentences = (list(itertools.chain(dataset_tweets_complessivo['Text_clean_1'])))
flat_list = [item for sublist in sentences for item in sublist]
flat_list[:30]

c = Counter(flat_list)
c.most_common(15)

[('tyson', 358989),
 ('jones', 253815),
 ('mike', 220294),
 ('roy', 165119),
 ('fight', 130319),
 ('#tysonvsjones', 110503),
 ('jr', 100962),
 ('live', 83864),
 ('#tysonjones', 75749),
 ('nate', 61672),
 ('tonight', 49362),
 ('robinson', 48506),
 ('like', 44254),
 ('stream', 41806),
 ('paul', 41060)]

# PRE-PROCESSING DATASET COMPLESSIVO #

In [ ]:
dataset_tweets_complessivo['Date_time_italy'] = pd.to_datetime(dataset_tweets_complessivo['Date_time_italy'])

In [ ]:
#Andiamo a creare un nuovo datetime che 
dataset_tweets_complessivo['Date_time_losangeles'] = pd.DatetimeIndex(dataset_tweets_complessivo['Date_time_italy']) - timedelta(hours=9)
dataset_tweets_complessivo.head(2)

,_id,Screen_name,Name,Text,Followers,Number_Retweet,Position,Date_time_italy,Hashtags,hashtags,Text_clean_1,Date_time_losangeles
0,5fc10d706b90cd1acfac6ad0,choholla33,BOSS UP,RT @ESPNRingside: Iron @MikeTyson has one of t...,280,0,None,2020-11-27 14:30:03,"[{'indices': [106, 117], 'text': 'TysonJones'}]",[TysonJones],"[@espnringside, iron, @miketyson, one, memorab...",2020-11-27 05:30:03
1,5fc10d706b90cd1acfac6ad1,Stars4P,Stars4Patriots,RT @DonaldJTrumpJr: This is going to be my wee...,15661,0,https://wego.social/Alleanna,2020-11-27 14:30:03,[],[],"[@donaldjtrumpjr, going, weekend, guilty, plea...",2020-11-27 05:30:03


In [ ]:
dataset_tweets_complessivo_preproc = dataset_tweets_complessivo

In [ ]:
#Effetto con il comando explode per 
dataset_tweets_complessivo_preproc = dataset_tweets_complessivo_preproc.explode("hashtags")
dataset_tweets_complessivo_preproc.head(5)

,_id,Screen_name,Name,Text,Followers,Number_Retweet,Position,Date_time_italy,Hashtags,hashtags,Text_clean_1,Date_time_losangeles
0,5fc10d706b90cd1acfac6ad0,choholla33,BOSS UP,RT @ESPNRingside: Iron @MikeTyson has one of t...,280,0,None,2020-11-27 14:30:03,"[{'indices': [106, 117], 'text': 'TysonJones'}]",TysonJones,"[@espnringside, iron, @miketyson, one, memorab...",2020-11-27 05:30:03
1,5fc10d706b90cd1acfac6ad1,Stars4P,Stars4Patriots,RT @DonaldJTrumpJr: This is going to be my wee...,15661,0,https://wego.social/Alleanna,2020-11-27 14:30:03,[],NaN,"[@donaldjtrumpjr, going, weekend, guilty, plea...",2020-11-27 05:30:03
2,5fc10d746b90cd1acfac6ad2,rywoo,Ryan Wolfe,This guy is gets more annoying the more I see him,208,0,None,2020-11-27 14:30:07,[],NaN,"[guy, get, annoying, see]",2020-11-27 05:30:07
3,5fc10d746b90cd1acfac6ad3,psdanks,Paul,RT @BTSportBoxing: One more sleep until @MikeT...,381,0,"Urmston, Manchester",2020-11-27 14:30:07,"[{'indices': [67, 78], 'text': 'TysonJones'}]",TysonJones,"[@btsportboxing, one, sleep, @miketyson, fight...",2020-11-27 05:30:07
4,5fc10d7a6b90cd1acfac6ad4,Vinchy1,Vinchy,RT @DonaldJTrumpJr: This is going to be my wee...,498,0,Los Angeles CA.,2020-11-27 14:30:12,[],NaN,"[@donaldjtrumpjr, going, weekend, guilty, plea...",2020-11-27 05:30:12


In [ ]:
#[ESTRAZIONE TWEETS IN BASE AD HASHTAGS]
#COSTRUISCO I DATASETS CON TWEETS LEGATI A TYSON E JONES DISTINTAMENTE:
dataset_hashtags_tyson = dataset_tweets_complessivo_preproc.loc[(dataset_tweets_complessivo_preproc['hashtags'] == "MikeTyson") | (dataset_tweets_complessivo_preproc['hashtags'] == "Tyson" ) | (dataset_tweets_complessivo_preproc['hashtags'] == "IronMike" )]
print("Nel dataset con duplicati, tweets con hashtags #MikeTyson, #Tyson e #IronMike, vi sono " + str(len(dataset_hashtags_tyson.index)) + " tweets")
dataset_hashtags_jones = dataset_tweets_complessivo_preproc.loc[(dataset_tweets_complessivo_preproc['hashtags'] == "Jones") | (dataset_tweets_complessivo_preproc['hashtags'] == "RoyJones" ) | (dataset_tweets_complessivo_preproc['hashtags'] == "jonesjr" ) | (dataset_tweets_complessivo_preproc['hashtags'] == "royjonesjr" )]
print("Nel dataset con duplicati, tweets con hashtags #Jones, #RoyJones e #royjonesjr, vi sono " + str(len(dataset_hashtags_jones.index)) + " tweets")

Nel dataset con duplicati, tweets con hashtags #MikeTyson, #Tyson e #IronMike, vi sono 9531 tweets
Nel dataset con duplicati, tweets con hashtags #Jones, #RoyJones e #royjonesjr, vi sono 702 tweets


In [ ]:
dataset_hashtags_tyson_nod = dataset_hashtags_tyson.drop_duplicates(['_id'], keep="last")
print("[FINALE HASHTAGS] Nel dataset con hastags legati a Tyson, vi sono: " + str(len(dataset_hashtags_tyson_nod.index)) + " tweets")
dataset_hashtags_jones_nod = dataset_hashtags_jones.drop_duplicates(['_id'], keep="last")
print("[FINALE HASHTAGS] Nel dataset con hashtags legati a Jones, vi sono: " + str(len(dataset_hashtags_jones_nod.index)) + " tweets")

[FINALE HASHTAGS] Nel dataset con hastags legati a Tyson, vi sono: 9120 tweets
[FINALE HASHTAGS] Nel dataset con hashtags legati a Jones, vi sono: 698 tweets


In [ ]:
#[ESTRAZIONE TWEETS IN BASE A PAROLE NEI TWEETS]
parole_testo_tyson =['Tyson', 'MikeTyson', 'IronMike', 'tyson']
def matcherTyson(x):
    for i in parole_testo_tyson:
        if i in x:
            return i
    else:
        return np.nan

parole_testo_jones =['Jones', 'Roy', 'jones', 'roy']
def matcherJones(x):
    for i in parole_testo_jones:
        if i in x:
            return i
    else:
        return np.nan
    
parole_testo_totale =['Jones', 'Roy', 'jones', 'roy', 'Tyson', 'IronMike', 'tyson']
def matcherTotale(x):
    for i in parole_testo_totale:
        if i in x:
            return i
    else:
        return np.nan

dataset_tweets_complessivo_preproc['Match_parola_Tyson'] = dataset_tweets_complessivo_preproc['hashtags'].apply(matcherTyson)
dataset_tweets_complessivo_preproc['Match_parola_Jones'] = dataset_tweets_complessivo_preproc['hashtags'].apply(matcherJones)
dataset_tweets_complessivo_preproc['Match_parola'] = dataset_tweets_complessivo_preproc['hashtags'].apply(matcherTotale)
dataset_tweets_complessivo_preproc_parole = dataset_tweets_complessivo_preproc[dataset_tweets_complessivo_preproc['Match_parola'].notna()]

#[TYSON]
dataset_parole_tyson = dataset_tweets_complessivo_preproc_parole.loc[ (dataset_tweets_complessivo_preproc_parole['Match_parola_Tyson'] == "Tyson") | (dataset_tweets_complessivo_preproc_parole['Match_parola_Tyson'] == "Mike") | (dataset_tweets_complessivo_preproc_parole['Match_parola_Tyson'] == "IronMike") | (dataset_tweets_complessivo_preproc_parole['Match_parola_Tyson'] == "tyson")]
dataset_parole_tyson = dataset_parole_tyson.loc[ (dataset_parole_tyson['Match_parola'] == "Tyson") | (dataset_parole_tyson['Match_parola'] == "tyson") | (dataset_parole_tyson['Match_parola'] == "IronMike")]
dataset_parole_tyson = dataset_parole_tyson[dataset_parole_tyson['Match_parola_Jones'].isnull()]
#Dataset nel quale sono contenuti i tweets dove sono contenute le parole riferite a Tyson nel testo:
dataset_parole_tyson_nod = dataset_parole_tyson.drop_duplicates(['_id'], keep="last")
print("[FINALE PAROLE] Nel dataset con tweets riferiti a Tyson, vi sono: " + str(len(dataset_parole_tyson_nod.index)) + " tweets")

#BIDEN:
dataset_parole_jones = dataset_tweets_complessivo_preproc_parole.loc[(dataset_tweets_complessivo_preproc_parole['Match_parola_Jones'] == "Jones") | (dataset_tweets_complessivo_preproc_parole['Match_parola_Jones'] == "Roy") | (dataset_tweets_complessivo_preproc_parole['Match_parola_Jones'] == "jones") | (dataset_tweets_complessivo_preproc_parole['Match_parola_Jones'] == "roy")]
dataset_parole_jones = dataset_parole_jones.loc[(dataset_parole_jones['Match_parola'] == "Jones") | (dataset_parole_jones['Match_parola'] == "Roy") | (dataset_parole_jones['Match_parola'] == "roy") | (dataset_parole_jones['Match_parola'] == "jones")]
dataset_parole_jones = dataset_parole_jones[dataset_parole_jones['Match_parola_Tyson'].isnull()]
#Dataset nel quale sono contenuti i tweets dove sono contenute le parole riferite a trump nel testo:
dataset_parole_jones_nod = dataset_parole_jones.drop_duplicates(['_id'], keep="last")
print("[FINALE PAROLE] Nel dataset con tweets legati a Jones, vi sono: " + str(len(dataset_parole_jones_nod.index)) + " tweets")

[FINALE PAROLE] Nel dataset con tweets riferiti a Tyson, vi sono: 193608 tweets
[FINALE PAROLE] Nel dataset con tweets legati a Jones, vi sono: 103239 tweets


In [ ]:
#[CREAZIONE DATASET FINALE CON TWEETS DEL 3 NOVEMBRE RIFERITI A TRUMP]
#UNIONE UNICO DATASET PER TRUMP E BIDEN CON I TWEETS CHE CONTENGONO HASHTAGS E PAROLE RIFERITI A LORO DUE:
#TRUMP
dataset_tyson = pd.concat([dataset_parole_tyson_nod, dataset_hashtags_tyson_nod])
dataset_tyson_def = dataset_tyson.drop_duplicates(['_id'], keep="last")
print("Nel dataset finale relativo a Tyson, vi sono:" + str(len(dataset_tyson_def.index)) + " tweets")

#BIDEN:
dataset_jones = pd.concat([dataset_parole_jones_nod, dataset_hashtags_jones_nod])
dataset_jones_def = dataset_jones.drop_duplicates(['_id'], keep="last")
print("Nel dataset finale relativo a Jones, vi sono:" + str(len(dataset_jones_def.index)) + " tweets")

Nel dataset finale relativo a Tyson, vi sono:199658 tweets
Nel dataset finale relativo a Jones, vi sono:103875 tweets


In [ ]:
dataset_jones_def['Date_time_italy'] = pd.to_datetime(dataset_jones_def['Date_time_italy'])
dataset_tyson_def['Date_time_italy'] = pd.to_datetime(dataset_tyson_def['Date_time_italy'])
dataset_jones_def = dataset_jones_def.sort_values(by = 'Date_time_italy')
dataset_tyson_def = dataset_tyson_def.sort_values(by = 'Date_time_italy')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#CREAZIONE DEL DATASET PER VISUALIZZAZIONE INTERMEDIA 
#unisci i dataset, aggiungi una colonna di count per ogni riga, aggiungi una colonna di count per ogni riga dove la parola matchata è tyson/jones 
#aggiungi una colonna di count dove l'hashtag è tyson/jones
dataset_viz = pd.concat([dataset_jones_def,dataset_tyson_def])
dataset_viz['count'] = 1
dataset_viz = dataset_viz.sort_values(by = 'Date_time_italy')
def t(row):
    if row['Match_parola'] == 'Tyson':
        val = 1
    else:
        val = 0
    return val

def j(row):
    if row['Match_parola'] == 'Jones':
        val = 1
    else:
        val = 0
    return val
  
  
dataset_viz['matching_parola_tyson'] = dataset_viz.apply(t, axis = 1)
dataset_viz['matching_parola_jones'] = dataset_viz.apply(j, axis = 1)

def jtest(row):
    if row['hashtags'] == 'Jones':
        val = 1
    elif row['hashtags'] == 'RoyJones':
        val=1
    elif row['hashtags'] == 'jonesjr':
        val = 1
    elif row['hashtags'] == 'royjonesjr':
        val = 1
    else:
        val = 0
    return val

def ttest(row):
    if row['hashtags'] == 'Tyson':
        val = 1
    elif row['hashtags'] == 'MikeTyson':
        val=1
    elif row['hashtags'] == 'IronMike':
        val = 1
    else:
        val = 0
    return val

dataset_viz['matching_hash_jones'] = dataset_viz.apply(jtest, axis = 1)
dataset_viz['matching_hash_tyson'] = dataset_viz.apply(ttest, axis = 1)
dataset_viz = dataset_viz.drop(['_id','Text'], axis = 1)
dataset_viz = pd.DataFrame(dataset_viz)
dataset_viz.to_csv (r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/ESPERIMENTITABLEAU/viztysonjones2.csv', index = False, header=True)

In [ ]:
#ESTRAZIONE PRE-INCONTRO TYSON :
tweets_preincontro_tyson = (dataset_tyson_def['Date_time_italy'] > "2020-11-27 10:36:47") & (dataset_tyson_def['Date_time_italy'] <= "2020-11-29 05:58:30")
tweets_preincontro_tyson = dataset_tyson_def.loc[tweets_preincontro_tyson]
tweets_preincontro_tyson = tweets_preincontro_tyson.reset_index(drop = True)
print("Tweets pre-incontro riferiti a Tyson: ", len(tweets_preincontro_tyson))

#ESTRAZIONE PRE-INCONTRO JONES :
tweets_preincontro_jones = (dataset_jones_def['Date_time_italy'] > "2020-11-27 10:36:47") & (dataset_jones_def['Date_time_italy'] <= "2020-11-29 05:58:30")
tweets_preincontro_jones = dataset_jones_def.loc[tweets_preincontro_jones]
tweets_preincontro_jones = tweets_preincontro_jones.reset_index(drop = True)
print("Tweets pre-incontro riferiti a Jones: ", len(tweets_preincontro_jones))

Tweets pre-incontro riferiti a Tyson:  102452
Tweets pre-incontro riferiti a Jones:  46716


In [ ]:
#Estrazione dataset tweets durante incontro TYSON:
tweets_incontro_tyson = (dataset_tyson_def['Date_time_italy'] > "2020-11-29 05:58:31") & (dataset_tyson_def['Date_time_italy'] < "2020-11-29 06:31:00")
tweets_incontro_tyson = dataset_tyson_def.loc[tweets_incontro_tyson]
tweets_incontro_tyson = tweets_incontro_tyson.reset_index(drop = True)
print("Tweets incontro riferiti a Tyson: ", len(tweets_incontro_tyson))

tweets_incontro_jones = (dataset_jones_def['Date_time_italy'] > "2020-11-29 05:58:31") & (dataset_jones_def['Date_time_italy'] < "2020-11-29 06:31:00")
tweets_incontro_jones = dataset_jones_def.loc[tweets_incontro_jones]
tweets_incontro_jones = tweets_incontro_jones.reset_index(drop = True)
print("Tweets incontro riferiti a Jones: ", len(tweets_incontro_jones))

Tweets incontro riferiti a Tyson:  28993
Tweets incontro riferiti a Jones:  9867


In [ ]:
#Vado a creare una colonna ROUND con tutti valori pari a zero, per poi andre ad integrare:
tweets_incontro_tyson.insert(0, 'ROUND', 0)
tweets_incontro_jones.insert(0, 'ROUND', 0)

ValueError: cannot insert ROUND, already exists

In [ ]:
#VADO A COSTRUIRE GLI INTERVALLI DEI TEMPI:
mask_1_t = (tweets_incontro_tyson['Date_time_italy'] > "2020-11-29 05:58:00") & (tweets_incontro_tyson['Date_time_italy'] < "2020-11-29 06:01:00")
mask_2_t = (tweets_incontro_tyson['Date_time_italy'] > "2020-11-29 06:01:01") & (tweets_incontro_tyson['Date_time_italy'] < "2020-11-29 06:04:00")
mask_3_t = (tweets_incontro_tyson['Date_time_italy'] > "2020-11-29 06:04:01") & (tweets_incontro_tyson['Date_time_italy'] < "2020-11-29 06:07:00")
mask_4_t = (tweets_incontro_tyson['Date_time_italy'] > "2020-11-29 06:07:01") & (tweets_incontro_tyson['Date_time_italy'] < "2020-11-29 06:10:00")
mask_5_t = (tweets_incontro_tyson['Date_time_italy'] > "2020-11-29 06:10:01") & (tweets_incontro_tyson['Date_time_italy'] < "2020-11-29 06:13:00")
mask_6_t = (tweets_incontro_tyson['Date_time_italy'] > "2020-11-29 06:13:01") & (tweets_incontro_tyson['Date_time_italy'] < "2020-11-29 06:16:00")
mask_7_t = (tweets_incontro_tyson['Date_time_italy'] > "2020-11-29 06:16:01") & (tweets_incontro_tyson['Date_time_italy'] < "2020-11-29 06:19:00")
mask_8_t = (tweets_incontro_tyson['Date_time_italy'] > "2020-11-29 06:19:01") & (tweets_incontro_tyson['Date_time_italy'] < "2020-11-29 06:22:00")

mask_1_j = (tweets_incontro_jones['Date_time_italy'] > "2020-11-29 05:58:00") & (tweets_incontro_jones['Date_time_italy'] < "2020-11-29 06:01:00")
mask_2_j = (tweets_incontro_jones['Date_time_italy'] > "2020-11-29 06:01:01") & (tweets_incontro_jones['Date_time_italy'] < "2020-11-29 06:04:00")
mask_3_j = (tweets_incontro_jones['Date_time_italy'] > "2020-11-29 06:04:01") & (tweets_incontro_jones['Date_time_italy'] < "2020-11-29 06:07:00")
mask_4_j = (tweets_incontro_jones['Date_time_italy'] > "2020-11-29 06:07:01") & (tweets_incontro_jones['Date_time_italy'] < "2020-11-29 06:10:00")
mask_5_j = (tweets_incontro_jones['Date_time_italy'] > "2020-11-29 06:10:01") & (tweets_incontro_jones['Date_time_italy'] < "2020-11-29 06:13:00")
mask_6_j = (tweets_incontro_jones['Date_time_italy'] > "2020-11-29 06:13:01") & (tweets_incontro_jones['Date_time_italy'] < "2020-11-29 06:16:00")
mask_7_j = (tweets_incontro_jones['Date_time_italy'] > "2020-11-29 06:16:01") & (tweets_incontro_jones['Date_time_italy'] < "2020-11-29 06:19:00")
mask_8_j = (tweets_incontro_jones['Date_time_italy'] > "2020-11-29 06:19:01") & (tweets_incontro_jones['Date_time_italy'] < "2020-11-29 06:22:00")

In [ ]:
#Inserisco per le fasce orarie il valore del ROUND
tweets_incontro_tyson["ROUND"].loc[mask_1_t] = 1
tweets_incontro_tyson["ROUND"].loc[mask_2_t] = 2
tweets_incontro_tyson["ROUND"].loc[mask_3_t] = 3
tweets_incontro_tyson["ROUND"].loc[mask_4_t] = 4
tweets_incontro_tyson["ROUND"].loc[mask_5_t] = 5
tweets_incontro_tyson["ROUND"].loc[mask_6_t] = 6
tweets_incontro_tyson["ROUND"].loc[mask_7_t] = 7
tweets_incontro_tyson["ROUND"].loc[mask_8_t] = 8

tweets_incontro_jones["ROUND"].loc[mask_1_j] = 1
tweets_incontro_jones["ROUND"].loc[mask_2_j] = 2
tweets_incontro_jones["ROUND"].loc[mask_3_j] = 3
tweets_incontro_jones["ROUND"].loc[mask_4_j] = 4
tweets_incontro_jones["ROUND"].loc[mask_5_j] = 5
tweets_incontro_jones["ROUND"].loc[mask_6_j] = 6
tweets_incontro_jones["ROUND"].loc[mask_7_j] = 7
tweets_incontro_jones["ROUND"].loc[mask_8_j] = 8

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
#[INTEGRAZIONE]
datasetcompleto_incontro_tyson = pd.merge(tweets_incontro_tyson, incontro_jab_landed, how='left')
datasetcompleto_incontro_jones = pd.merge(tweets_incontro_jones, incontro_jab_landed, how='left')

datasetcompleto_incontro_tyson = pd.merge(datasetcompleto_incontro_tyson, incontro_punches_total, how='left')
datasetcompleto_incontro_jones = pd.merge(datasetcompleto_incontro_jones, incontro_punches_total, how='left')

datasetcompleto_incontro_tyson = pd.merge(datasetcompleto_incontro_tyson, incontro_powerpunches_landed, how='left')
datasetcompleto_incontro_jones = pd.merge(datasetcompleto_incontro_jones, incontro_powerpunches_landed, how='left')

In [ ]:
#tweets_incontro_tyson['ROUND'] = np.where((tweets_incontro_tyson['Date_time_italy'] >= '2020-11-29 06:04:31') & (tweets_incontro_tyson['Date_time_italy'] <= '2020-11-29 06:31:00'), 2, 10)
#tweets_incontro_tyson['ROUND'] = tweets_incontro_tyson['Date_time_italy'].apply(lambda x: "2" if (x >= '2020-11-29 06:04:31' and x <= '2020-11-29 06:33:00'))
#tweets_incontro_tyson.loc[tweets_incontro_tyson['Date_time_italy'] >= '2020-11-29 06:04:31' & tweets_incontro_tyson['Date_time_italy'] <= '2020-11-29 06:31:00', tweets_incontro_tyson['ROUND']] = 10


In [ ]:
#Estrazione dataset tweets dopo incontro TYSON:
tweets_afterincontro_tyson = (dataset_tyson_def['Date_time_italy'] > "2020-11-29 06:23:01") & (dataset_tyson_def['Date_time_italy'] <= "2020-12-05 06:31:00")
tweets_afterincontro_tyson = dataset_tyson_def.loc[tweets_afterincontro_tyson]
tweets_afterincontro_tyson = tweets_afterincontro_tyson.reset_index(drop = True)
print("Tweest riferiti a Tyson nel dopo incontro: ", len(tweets_afterincontro_tyson))

#Estrazione dataset tweets durante incontro JONES:
tweets_afterincontro_jones = (dataset_jones_def['Date_time_italy'] > "2020-11-29 06:23:01") & (dataset_jones_def['Date_time_italy'] <= "2020-12-05 06:31:00")
tweets_afterincontro_jones = dataset_jones_def.loc[tweets_afterincontro_jones]
tweets_afterincontro_jones = tweets_afterincontro_jones.reset_index(drop = True)
print("Tweest riferiti a Jones nel dopo incontro: ", len(tweets_afterincontro_jones))

Tweest riferiti a Tyson nel dopo incontro:  74918
Tweest riferiti a Jones nel dopo incontro:  50502


In [ ]:
#AGGIUNGO INFORMAZIONI AL DATASET COMPLESSIVO DEI TWEETS:

In [ ]:
dataset_tweets_complessivo.insert(0, 'ROUND', 0)
dataset_tweets_complessivo.head()

ValueError: cannot insert ROUND, already exists

In [ ]:
mask_1_comp = (dataset_tweets_complessivo['Date_time_italy'] > "2020-11-29 05:58:00") & (dataset_tweets_complessivo['Date_time_italy'] < "2020-11-29 06:01:00")
mask_2_comp = (dataset_tweets_complessivo['Date_time_italy'] > "2020-11-29 06:01:01") & (dataset_tweets_complessivo['Date_time_italy'] < "2020-11-29 06:04:00")
mask_3_comp = (dataset_tweets_complessivo['Date_time_italy'] > "2020-11-29 06:04:01") & (dataset_tweets_complessivo['Date_time_italy'] < "2020-11-29 06:07:00")
mask_4_comp = (dataset_tweets_complessivo['Date_time_italy'] > "2020-11-29 06:07:01") & (dataset_tweets_complessivo['Date_time_italy'] < "2020-11-29 06:10:00")
mask_5_comp = (dataset_tweets_complessivo['Date_time_italy'] > "2020-11-29 06:10:01") & (dataset_tweets_complessivo['Date_time_italy'] < "2020-11-29 06:13:00")
mask_6_comp = (dataset_tweets_complessivo['Date_time_italy'] > "2020-11-29 06:13:01") & (dataset_tweets_complessivo['Date_time_italy'] < "2020-11-29 06:16:00")
mask_7_comp = (dataset_tweets_complessivo['Date_time_italy'] > "2020-11-29 06:16:01") & (dataset_tweets_complessivo['Date_time_italy'] < "2020-11-29 06:19:00")
mask_8_comp = (dataset_tweets_complessivo['Date_time_italy'] > "2020-11-29 06:19:01") & (dataset_tweets_complessivo['Date_time_italy'] < "2020-12-20 06:22:00")

#Inserisco per le fasce orarie il valore del ROUND
dataset_tweets_complessivo["ROUND"].loc[mask_1_comp] = 1
dataset_tweets_complessivo["ROUND"].loc[mask_2_comp] = 2
dataset_tweets_complessivo["ROUND"].loc[mask_3_comp] = 3
dataset_tweets_complessivo["ROUND"].loc[mask_4_comp] = 4
dataset_tweets_complessivo["ROUND"].loc[mask_5_comp] = 5
dataset_tweets_complessivo["ROUND"].loc[mask_6_comp] = 6
dataset_tweets_complessivo["ROUND"].loc[mask_7_comp] = 7
dataset_tweets_complessivo["ROUND"].loc[mask_8_comp] = 8

In [ ]:
#[TEST INTEGRAZIONE JAB LANDED DATASET COMPLESSIVO]
datasetcompleto_integrazione = pd.merge(dataset_tweets_complessivo, incontro_jab_landed, how='left')
datasetcompleto_integrazione = pd.merge(datasetcompleto_integrazione, incontro_punches_total, how='left')
datasetcompleto_integrazione = pd.merge(datasetcompleto_integrazione, incontro_powerpunches_landed, how='left')
datasetcompleto_integrazione = pd.merge(datasetcompleto_integrazione, tabellino_incontro_t, how='left')
datasetcompleto_integrazione.tail(3)

,ROUND,_id,Screen_name,Name,Text,Followers,Number_Retweet,Position,Date_time_italy,Hashtags,...,tot_punch_tirati_tyson,tot_punch_tirati_jones,CUMSUM_power_tirati_tyson,CUMSUM_power_tirati_jones,CUMSUM_pugni_2_landed_tyson,CUMSUM_pugni_2_landed_jones,TYSON_punteggio_sing,JONES_punteggio_sing,CUMSUM_punteggio_tyson,CUMSUM_punteggio_jones
767470,8,5fc35167fec2296bd50ec512,JasmineBagley_,Spicy Jazzy,RT @ScottieBeam: Mike Tyson: we gonna do this ...,708,0,"Dallas, TX",2020-11-29 08:44:34,[],...,11.0,4.0,137.0,151.0,57.0,28.0,9.0,10.0,76.0,76.0
767471,8,5fc35167fec2296bd50ec513,TH_Adrian_,𝓣𝓱𝓮 𝓦𝓮𝓼𝓽,That's cap,66,0,None,2020-11-29 08:44:34,[],...,11.0,4.0,137.0,151.0,57.0,28.0,9.0,10.0,76.0,76.0
767472,8,5fc35168fec2296bd50ec514,Mooschbenari,Moosch,RT @RexChapman: Preview of the Mike Tyson vs. ...,18,0,"London, England",2020-11-29 08:44:34,[],...,11.0,4.0,137.0,151.0,57.0,28.0,9.0,10.0,76.0,76.0


In [ ]:
datasetcompleto_integrazione.to_csv(r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/datasetcompletointegrato.csv')

# EXPLORATION DATASET COMPLESSIVO #

In [ ]:
#Vado a fare un'analisi dei tweets per le date presenti nei tweets:
conteggioorario_complessivo = dataset_tweets_complessivo.groupby('Date_time_italy').size().reset_index(name='count')
conteggioorario_complessivo = pd.DataFrame(conteggioorario_complessivo)
conteggioorario_complessivo_ordinato_count = conteggioorario_complessivo.sort_values('count', ascending=False)
conteggioorario_complessivo_ordinato_ora = conteggioorario_complessivo.sort_values('Date_time_italy', ascending=True)
conteggioorario_complessivo_ordinato_count

,Date_time_italy,count
82640,2020-11-29 05:56:46,81
83943,2020-11-29 06:18:29,75
84430,2020-11-29 06:26:36,74
83572,2020-11-29 06:12:18,73
84715,2020-11-29 06:31:21,72
...,...,...
24792,2020-11-28 09:43:45,1
108771,2020-11-29 17:09:06,1
24793,2020-11-28 09:43:46,1
24794,2020-11-28 09:43:47,1


In [ ]:
conteggioorario_complessivo_ordinato_count = conteggioorario_complessivo_ordinato_count.reset_index(drop = True)

conteggioorario_complessivo = conteggioorario_complessivo.reset_index(drop = True)

In [ ]:
conteggioorario_complessivo.head()

,Date_time_italy,count
0,2020-11-27 14:30:03,2
1,2020-11-27 14:30:07,2
2,2020-11-27 14:30:12,1
3,2020-11-27 14:30:18,1
4,2020-11-27 14:30:19,1


In [ ]:
range_datetime_interesse = (conteggioorario_complessivo['Date_time_italy'] > '2020-11-28 15:00:00') & (conteggioorario_complessivo['Date_time_italy'] <= '2020-11-29 13:00:00')
conteggioorario_interesse = conteggioorario_complessivo.loc[range_datetime_interesse]

In [ ]:
conteggioorario_interesse.to_csv(r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/ESPERIMENTITABLEAU/conteggioorario_interesse.csv')
conteggioorario_complessivo.to_csv(r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/ESPERIMENTITABLEAU/conteggioorario_complessivo.csv')

# SENTIMENT TWEETS PRE INCONTRO #

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from pylab import rcParams


In [ ]:
from afinn import Afinn
afinn = Afinn(language='en')


In [ ]:
def rescale3(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -17, 12

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output

def rescale1(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -12, 11

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output

def rescale2(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -14, 10

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output

def rescale33(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -12, 10

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output

def rescale5(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -17, 8

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output

def rescale6(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -10, 10

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output

def rescale7(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -12, 12

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output    
  
def rescale8(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -14, 12

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output    

def rescaleafter(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -24, 20

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output     
  
def rescalepre(values, new_min = 8, new_max = 10):
    output = []
    old_min, old_max = -30, 21

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output  

In [ ]:
tweets_preincontro_tyson['Text_clean_1']=tweets_preincontro_tyson['Text_clean_1'].apply(str)
tweets_preincontro_jones['Text_clean_1']=tweets_preincontro_jones['Text_clean_1'].apply(str)
tweets_preincontro_tyson['afinn'] = tweets_preincontro_tyson['Text_clean_1'].apply(afinn.score)
tweets_preincontro_jones['afinn'] = tweets_preincontro_jones['Text_clean_1'].apply(afinn.score)


In [ ]:
print("Describe di Afinn di Tyson PRE-INCONTRO:")
print(tweets_preincontro_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones PRE-INCONTRO:")
print(tweets_preincontro_jones['afinn'].describe())


Describe di Afinn di Tyson PRE-INCONTRO:
count    102452.000000
mean          0.163413
std           2.750718
min         -30.000000
25%          -1.000000
50%           0.000000
75%           1.000000
max          21.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones PRE-INCONTRO:
count    46716.000000
mean         0.488484
std          2.314688
min        -20.000000
25%          0.000000
50%          0.000000
75%          2.000000
max         17.000000
Name: afinn, dtype: float64


In [ ]:
tweets_preincontro_tyson['Afinnscore'] = rescalepre(tweets_preincontro_tyson['afinn'])
tweets_preincontro_jones['Afinnscore'] = rescalepre(tweets_preincontro_jones['afinn']) 
print('Describe afinn pre incontro Tyson')
print(tweets_preincontro_tyson['Afinnscore'].describe())
print('Describe afinn pre incontro Jones')
print(tweets_preincontro_jones['Afinnscore'].describe())


Describe afinn pre incontro Tyson
count    102452.000000
mean          9.182879
std           0.107871
min           8.000000
25%           9.137255
50%           9.176471
75%           9.215686
max          10.000000
Name: Afinnscore, dtype: float64
Describe afinn pre incontro Jones
count    46716.000000
mean         9.195627
std          0.090772
min          8.392157
25%          9.176471
50%          9.176471
75%          9.254902
max          9.843137
Name: Afinnscore, dtype: float64


In [ ]:
tweets_preincontro_tyson_test1 = (tweets_preincontro_tyson.groupby([pd.Grouper(key='Date_time_italy', freq='10min')]).agg({'Afinnscore':'mean', 'afinn':'mean'}))
tweets_preincontro_jones_test1 = (tweets_preincontro_jones.groupby([pd.Grouper(key='Date_time_italy', freq='10min')]).agg({'Afinnscore':'mean', 'afinn':'mean'}))
tweets_preincontro_tyson_test2 = (tweets_preincontro_tyson.groupby([pd.Grouper(key='Date_time_italy', freq='10min')]).agg({'Afinnscore':'std', 'afinn':'std'}))
tweets_preincontro_jones_test2 = (tweets_preincontro_jones.groupby([pd.Grouper(key='Date_time_italy', freq='10min')]).agg({'Afinnscore':'std', 'afinn':'std'}))

In [ ]:
tweets_preincontro_tyson_test1 = tweets_preincontro_tyson_test1.reset_index()
tweets_preincontro_jones_test1 = tweets_preincontro_jones_test1.reset_index()
tweets_preincontro_tyson_test2 = tweets_preincontro_tyson_test2.reset_index()
tweets_preincontro_jones_test2 = tweets_preincontro_jones_test2.reset_index()

In [ ]:
tweets_preincontro_tyson_test1 = tweets_preincontro_tyson_test1.rename(columns={"Afinnscore": "Afinnscore_mean_tyson", "afinn": "afinn_mean_tyson"})
tweets_preincontro_jones_test1 = tweets_preincontro_jones_test1.rename(columns={"Afinnscore": "Afinnscore_mean_jones", "afinn": "afinn_mean_jones"})
tweets_preincontro_tyson_test2 = tweets_preincontro_tyson_test2.rename(columns={"Afinnscore": "Afinnscore_std_tyson", "afinn": "afinn_std_tyson"})
tweets_preincontro_jones_test2 = tweets_preincontro_jones_test2.rename(columns={"Afinnscore": "Afinnscore_std_jones", "afinn": "afinn_std_jones"})

In [ ]:
tweets_preincontro_andamentisentiment = pd.merge(tweets_preincontro_tyson_test1, tweets_preincontro_jones_test1, how='left')
tweets_preincontro_andamentisentiment = pd.merge(tweets_preincontro_andamentisentiment, tweets_preincontro_tyson_test2, how='left')
tweets_preincontro_andamentisentiment = pd.merge(tweets_preincontro_andamentisentiment, tweets_preincontro_jones_test2, how='left')

In [ ]:
tweets_preincontro_andamentisentiment['Afinnscore_mean_tyson'] = tweets_preincontro_andamentisentiment['Afinnscore_mean_tyson'].astype(str)
tweets_preincontro_andamentisentiment['Afinnscore_mean_jones'] = tweets_preincontro_andamentisentiment['Afinnscore_mean_jones'].astype(str)
tweets_preincontro_andamentisentiment['afinn_mean_tyson'] = tweets_preincontro_andamentisentiment['afinn_mean_tyson'].astype(str)
tweets_preincontro_andamentisentiment['afinn_mean_jones'] = tweets_preincontro_andamentisentiment['afinn_mean_jones'].astype(str)
tweets_preincontro_andamentisentiment['Afinnscore_std_tyson'] = tweets_preincontro_andamentisentiment['Afinnscore_std_tyson'].astype(str)
tweets_preincontro_andamentisentiment['Afinnscore_std_jones'] = tweets_preincontro_andamentisentiment['Afinnscore_std_jones'].astype(str)
tweets_preincontro_andamentisentiment['afinn_std_tyson'] = tweets_preincontro_andamentisentiment['afinn_std_tyson'].astype(str)
tweets_preincontro_andamentisentiment['afinn_std_jones'] = tweets_preincontro_andamentisentiment['afinn_std_jones'].astype(str)

In [ ]:
tweets_preincontro_andamentisentiment['Afinnscore_mean_tyson'] = tweets_preincontro_andamentisentiment['Afinnscore_mean_tyson'].apply(lambda x: x.replace('.', ','))
tweets_preincontro_andamentisentiment['Afinnscore_mean_jones'] = tweets_preincontro_andamentisentiment['Afinnscore_mean_jones'].apply(lambda x: x.replace('.', ','))
tweets_preincontro_andamentisentiment['afinn_mean_tyson'] = tweets_preincontro_andamentisentiment['afinn_mean_tyson'].apply(lambda x: x.replace('.', ','))
tweets_preincontro_andamentisentiment['afinn_mean_jones'] = tweets_preincontro_andamentisentiment['afinn_mean_jones'].apply(lambda x: x.replace('.', ','))
tweets_preincontro_andamentisentiment['Afinnscore_std_tyson'] = tweets_preincontro_andamentisentiment['Afinnscore_std_tyson'].apply(lambda x: x.replace('.', ','))
tweets_preincontro_andamentisentiment['Afinnscore_std_jones'] = tweets_preincontro_andamentisentiment['Afinnscore_std_jones'].apply(lambda x: x.replace('.', ','))
tweets_preincontro_andamentisentiment['afinn_std_tyson'] = tweets_preincontro_andamentisentiment['afinn_std_tyson'].apply(lambda x: x.replace('.', ','))
tweets_preincontro_andamentisentiment['afinn_std_jones'] = tweets_preincontro_andamentisentiment['afinn_std_jones'].apply(lambda x: x.replace('.', ','))

In [ ]:
tweets_preincontro_andamentisentiment.to_csv(r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/ESPERIMENTITABLEAU/tweets_preincontro_andamentisentiment.csv')

# SEMTIMENT INCONTRO #

### SENTIMENT ROUND 1 ###

In [ ]:
tweets_1round_tyson = datasetcompleto_incontro_tyson[datasetcompleto_incontro_tyson["ROUND"] == 1]
tweets_1round_tyson = tweets_1round_tyson.reset_index(drop = True)
print("Tweets presenti per il round 1 riferiti a Tyson:", len(tweets_1round_tyson))
tweets_1round_jones = datasetcompleto_incontro_jones[datasetcompleto_incontro_jones["ROUND"] == 1]
tweets_1round_jones = tweets_1round_jones.reset_index(drop = True)
print("Tweets presenti per il round 1 riferiti a Tyson:", len(tweets_1round_jones))


Tweets presenti per il round 1 riferiti a Tyson: 2383
Tweets presenti per il round 1 riferiti a Tyson: 692


In [ ]:
#[AFINN ROUND 1]
tweets_1round_tyson['Text_clean_1']=tweets_1round_tyson['Text_clean_1'].apply(str)
tweets_1round_jones['Text_clean_1']=tweets_1round_jones['Text_clean_1'].apply(str)
tweets_1round_jones['afinn'] = tweets_1round_jones['Text_clean_1'].apply(afinn.score)
tweets_1round_tyson['afinn'] = tweets_1round_tyson['Text_clean_1'].apply(afinn.score)


In [ ]:
print("Describe di Afinn di Tyson del ROUND 1:")
print(tweets_1round_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones del ROUND 1:")
print(tweets_1round_jones['afinn'].describe())

Describe di Afinn di Tyson del ROUND 1:
count    2383.000000
mean        0.010491
std         2.264865
min       -10.000000
25%        -1.000000
50%         0.000000
75%         0.000000
max        16.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones del ROUND 1:
count    692.000000
mean       0.364162
std        2.362953
min      -10.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        8.000000
Name: afinn, dtype: float64


In [ ]:
#AFINN RESCALE 1
tweets_1round_jones['Afinnscore'] = rescale1(tweets_1round_jones['afinn'])
tweets_1round_tyson['Afinnscore'] = rescale1(tweets_1round_tyson['afinn'])
print('votazione 1 round Tyson ')
print(tweets_1round_tyson['Afinnscore'].describe())
print('votazione 1 round Jones')
print(tweets_1round_jones['Afinnscore'].describe())

votazione 1 round Tyson 
count    2383.000000
mean        9.044391
std         0.196945
min         8.173913
25%         8.956522
50%         9.043478
75%         9.043478
max        10.434783
Name: Afinnscore, dtype: float64
votazione 1 round Jones
count    692.000000
mean       9.075145
std        0.205474
min        8.173913
25%        9.043478
50%        9.043478
75%        9.130435
max        9.739130
Name: Afinnscore, dtype: float64


In [ ]:
#[MEDIA e STD AFINN SIA NORMALE CHE AGGIUSTATA]
mean_afinn_round1_tyson = tweets_1round_tyson['afinn'].mean()
mean_afinn_round1_jones = tweets_1round_jones['afinn'].mean()
mean_afinnscore_round1_tyson = tweets_1round_tyson['Afinnscore'].mean()
mean_afinnscore_round1_jones = tweets_1round_jones['Afinnscore'].mean()
std_afinn_round1_tyson = tweets_1round_tyson['afinn'].std()
std_afinn_round1_jones = tweets_1round_jones['afinn'].std()
std_afinnscore_round1_tyson = tweets_1round_tyson['Afinnscore'].std()
std_afinnscore_round1_jones = tweets_1round_jones['Afinnscore'].std()


### SENTIMENT ROUND 2 ###

In [ ]:
tweets_2round_tyson = datasetcompleto_incontro_tyson[datasetcompleto_incontro_tyson["ROUND"] == 2]
tweets_2round_tyson = tweets_2round_tyson.reset_index(drop = True)
print("Tweets presenti per il round 2 riferiti a Tyson:", len(tweets_2round_tyson))
tweets_2round_jones = datasetcompleto_incontro_jones[datasetcompleto_incontro_jones["ROUND"] == 2]
tweets_2round_jones = tweets_2round_jones.reset_index(drop = True)
print("Tweets presenti per il round 2 riferiti a Tyson:", len(tweets_2round_jones))

Tweets presenti per il round 2 riferiti a Tyson: 3295
Tweets presenti per il round 2 riferiti a Tyson: 701


In [ ]:
#[AFINN SENTIMENT ROUND 2]
tweets_2round_tyson['Text_clean_1']=tweets_2round_tyson['Text_clean_1'].apply(str)
tweets_2round_jones['Text_clean_1']=tweets_2round_jones['Text_clean_1'].apply(str)
tweets_2round_jones['afinn'] = tweets_2round_jones['Text_clean_1'].apply(afinn.score)
tweets_2round_tyson['afinn'] = tweets_2round_tyson['Text_clean_1'].apply(afinn.score)


In [ ]:
print("Describe di Afinn di Tyson del ROUND 2:")
print(tweets_2round_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones del ROUND 2:")
print(tweets_2round_jones['afinn'].describe())

Describe di Afinn di Tyson del ROUND 2:
count    3295.000000
mean        0.060091
std         2.287748
min       -12.000000
25%        -1.000000
50%         0.000000
75%         1.000000
max        10.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones del ROUND 2:
count    701.000000
mean      -0.049929
std        2.634185
min      -10.000000
25%       -1.000000
50%        0.000000
75%        1.000000
max       10.000000
Name: afinn, dtype: float64


In [ ]:
#AFINN RESCALE ROUND 2
tweets_2round_jones['Afinnscore'] = rescale2(tweets_2round_jones['afinn'])
tweets_2round_tyson['Afinnscore'] = rescale2(tweets_2round_tyson['afinn'])
print('votazione 2 round tyson')
print(tweets_2round_tyson['Afinnscore'].describe())
print('votazione 2 round jones ')
print(tweets_2round_jones['Afinnscore'].describe())


votazione 2 round tyson
count    3295.000000
mean        9.171674
std         0.190646
min         8.166667
25%         9.083333
50%         9.166667
75%         9.250000
max        10.000000
Name: Afinnscore, dtype: float64
votazione 2 round jones 
count    701.000000
mean       9.162506
std        0.219515
min        8.333333
25%        9.083333
50%        9.166667
75%        9.250000
max       10.000000
Name: Afinnscore, dtype: float64


In [ ]:
#[MEDIA e STD AFINN SIA NORMALE CHE RESCALE]
mean_afinn_round2_tyson = tweets_2round_tyson['afinn'].mean()
mean_afinn_round2_jones = tweets_2round_jones['afinn'].mean()
mean_afinnscore_round2_tyson = tweets_2round_tyson['Afinnscore'].mean()
mean_afinnscore_round2_jones = tweets_2round_jones['Afinnscore'].mean()
std_afinn_round2_tyson = tweets_2round_tyson['afinn'].std()
std_afinn_round2_jones = tweets_2round_jones['afinn'].std()
std_afinnscore_round2_tyson = tweets_2round_tyson['Afinnscore'].std()
std_afinnscore_round2_jones = tweets_2round_jones['Afinnscore'].std()


### SENTIMENT ROUND 3 ###

In [ ]:
tweets_3round_tyson = datasetcompleto_incontro_tyson[datasetcompleto_incontro_tyson["ROUND"] == 3]
tweets_3round_tyson = tweets_3round_tyson.reset_index(drop = True)
print("Tweets presenti per il round 3 riferiti a Tyson:", len(tweets_3round_tyson))
tweets_3round_jones = datasetcompleto_incontro_jones[datasetcompleto_incontro_jones["ROUND"] == 3]
tweets_3round_jones = tweets_3round_jones.reset_index(drop = True)
print("Tweets presenti per il round 3 riferiti a Tyson:", len(tweets_3round_jones))


Tweets presenti per il round 3 riferiti a Tyson: 3129
Tweets presenti per il round 3 riferiti a Tyson: 790


In [ ]:
#AFINN SENTIMENT ROUND 3
tweets_3round_tyson['Text_clean_1']=tweets_3round_tyson['Text_clean_1'].apply(str)
tweets_3round_jones['Text_clean_1']=tweets_3round_jones['Text_clean_1'].apply(str)
tweets_3round_jones['afinn'] = tweets_3round_jones['Text_clean_1'].apply(afinn.score)
tweets_3round_tyson['afinn'] = tweets_3round_tyson['Text_clean_1'].apply(afinn.score)

In [ ]:
print("Describe di Afinn di Tyson del ROUND 3:")
print(tweets_3round_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones del ROUND 3:")
print(tweets_3round_jones['afinn'].describe())

Describe di Afinn di Tyson del ROUND 3:
count    3129.000000
mean        0.014062
std         2.469208
min       -10.000000
25%        -1.000000
50%         0.000000
75%         1.000000
max        11.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones del ROUND 3:
count    790.000000
mean       0.059494
std        2.289061
min      -14.000000
25%        0.000000
50%        0.000000
75%        2.000000
max        9.000000
Name: afinn, dtype: float64


In [ ]:
#AFINN SENTIMENT RESCALE ROUND 3
tweets_3round_jones['Afinnscore'] = rescale33(tweets_3round_jones['afinn'])
tweets_3round_tyson['Afinnscore'] = rescale33(tweets_3round_tyson['afinn'])
print('votazione 3 round Tyson ')
print(tweets_3round_tyson['Afinnscore'].describe())
print('votazione 3 round Jones')
print(tweets_3round_jones['Afinnscore'].describe())


votazione 3 round Tyson 
count    3129.000000
mean        9.092187
std         0.224473
min         8.181818
25%         9.000000
50%         9.090909
75%         9.181818
max        10.090909
Name: Afinnscore, dtype: float64
votazione 3 round Jones
count    790.000000
mean       9.096318
std        0.208096
min        7.818182
25%        9.090909
50%        9.090909
75%        9.272727
max        9.909091
Name: Afinnscore, dtype: float64


In [ ]:
#[MEDIA e STD AFINN SIA NORMALE CHE AGGIUSTATA]
mean_afinn_round3_tyson = tweets_3round_tyson['afinn'].mean()
mean_afinn_round3_jones = tweets_3round_jones['afinn'].mean()
mean_afinnscore_round3_tyson = tweets_3round_tyson['Afinnscore'].mean()
mean_afinnscore_round3_jones = tweets_3round_jones['Afinnscore'].mean()
std_afinn_round3_tyson = tweets_3round_tyson['afinn'].std()
std_afinn_round3_jones = tweets_3round_jones['afinn'].std()
std_afinnscore_round3_tyson = tweets_3round_tyson['Afinnscore'].std()
std_afinnscore_round3_jones = tweets_3round_jones['Afinnscore'].std()


### SENTIMENT ROUND 4 ###

In [ ]:
tweets_4round_tyson = datasetcompleto_incontro_tyson[datasetcompleto_incontro_tyson["ROUND"] == 4]
tweets_4round_tyson = tweets_4round_tyson.reset_index(drop = True)
print("Tweets presenti per il round 4 riferiti a Tyson:", len(tweets_4round_tyson))
tweets_4round_jones = datasetcompleto_incontro_jones[datasetcompleto_incontro_jones["ROUND"] == 4]
tweets_4round_jones = tweets_4round_jones.reset_index(drop = True)
print("Tweets presenti per il round 4 riferiti a Tyson:", len(tweets_4round_jones))


Tweets presenti per il round 4 riferiti a Tyson: 2570
Tweets presenti per il round 4 riferiti a Tyson: 797


In [ ]:
#AFINN SENTIMENT ROUND 4
tweets_4round_tyson['Text_clean_1']=tweets_4round_tyson['Text_clean_1'].apply(str)
tweets_4round_jones['Text_clean_1']=tweets_4round_jones['Text_clean_1'].apply(str)
tweets_4round_jones['afinn'] = tweets_4round_jones['Text_clean_1'].apply(afinn.score)
tweets_4round_tyson['afinn'] = tweets_4round_tyson['Text_clean_1'].apply(afinn.score)


In [ ]:
print("Describe di Afinn di Tyson del ROUND 4:")
print(tweets_4round_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones del ROUND 4:")
print(tweets_4round_jones['afinn'].describe())


Describe di Afinn di Tyson del ROUND 4:
count    2570.000000
mean        0.209728
std         2.565753
min        -9.000000
25%        -1.000000
50%         0.000000
75%         2.000000
max        10.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones del ROUND 4:
count    797.000000
mean       0.010038
std        2.393437
min      -12.000000
25%       -1.000000
50%        0.000000
75%        2.000000
max        8.000000
Name: afinn, dtype: float64


In [ ]:
#AFINN SENTIMENT ROUND 4 RESCALE
tweets_4round_jones['Afinnscore'] = rescale2(tweets_4round_jones['afinn'])
tweets_4round_tyson['Afinnscore'] = rescale2(tweets_4round_tyson['afinn'])
print('votazione 4 round Tyson ')
print(tweets_4round_tyson['Afinnscore'].describe())
print('votazione 4 round Jones')
print(tweets_4round_jones['Afinnscore'].describe())


votazione 4 round Tyson 
count    2570.000000
mean        9.184144
std         0.213813
min         8.416667
25%         9.083333
50%         9.166667
75%         9.333333
max        10.000000
Name: Afinnscore, dtype: float64
votazione 4 round Jones
count    797.000000
mean       9.167503
std        0.199453
min        8.166667
25%        9.083333
50%        9.166667
75%        9.333333
max        9.833333
Name: Afinnscore, dtype: float64


In [ ]:
#[MEDIA e STD AFINN SIA NORMALE CHE AGGIUSTATA]
mean_afinn_round4_tyson = tweets_4round_tyson['afinn'].mean()
mean_afinn_round4_jones = tweets_4round_jones['afinn'].mean()
mean_afinnscore_round4_tyson = tweets_4round_tyson['Afinnscore'].mean()
mean_afinnscore_round4_jones = tweets_4round_jones['Afinnscore'].mean()
std_afinn_round4_tyson = tweets_4round_tyson['afinn'].std()
std_afinn_round4_jones = tweets_4round_jones['afinn'].std()
std_afinnscore_round4_tyson = tweets_4round_tyson['Afinnscore'].std()
std_afinnscore_round4_jones = tweets_4round_jones['Afinnscore'].std()


### SENTIMENT ROUND 5 ###

In [ ]:
tweets_5round_tyson = datasetcompleto_incontro_tyson[datasetcompleto_incontro_tyson["ROUND"] == 5]
tweets_5round_tyson = tweets_5round_tyson.reset_index(drop = True)
print("Tweets presenti per il round 5 riferiti a Tyson:", len(tweets_5round_tyson))
tweets_5round_jones = datasetcompleto_incontro_jones[datasetcompleto_incontro_jones["ROUND"] == 5]
tweets_5round_jones = tweets_5round_jones.reset_index(drop = True)
print("Tweets presenti per il round 5 riferiti a Tyson:", len(tweets_5round_jones))

Tweets presenti per il round 5 riferiti a Tyson: 2113
Tweets presenti per il round 5 riferiti a Tyson: 904


In [ ]:
#AFINN SENTIMENT ROUND 5
tweets_5round_tyson['Text_clean_1']=tweets_5round_tyson['Text_clean_1'].apply(str)
tweets_5round_jones['Text_clean_1']=tweets_5round_jones['Text_clean_1'].apply(str)
tweets_5round_jones['afinn'] = tweets_5round_jones['Text_clean_1'].apply(afinn.score)
tweets_5round_tyson['afinn'] = tweets_5round_tyson['Text_clean_1'].apply(afinn.score)

In [ ]:
print("Describe di Afinn di Tyson del ROUND 5:")
print(tweets_5round_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones del ROUND 5:")
print(tweets_5round_jones['afinn'].describe())

Describe di Afinn di Tyson del ROUND 5:
count    2113.000000
mean       -0.033602
std         2.591746
min       -14.000000
25%        -1.000000
50%         0.000000
75%         1.000000
max        10.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones del ROUND 5:
count    904.000000
mean      -0.101770
std        2.157846
min      -13.000000
25%       -1.000000
50%        0.000000
75%        0.000000
max        9.000000
Name: afinn, dtype: float64


In [ ]:
#AFINN SENTIMENT ROUND 5 RESCALE
tweets_5round_jones['Afinnscore'] = rescale5(tweets_5round_jones['afinn'])
tweets_5round_tyson['Afinnscore'] = rescale5(tweets_5round_tyson['afinn'])
print('votazione 5 round Tyson ')
print(tweets_5round_tyson['Afinnscore'].describe())
print('votazione 5 round Jones')
print(tweets_5round_jones['Afinnscore'].describe())


votazione 5 round Tyson 
count    2113.000000
mean        9.357312
std         0.207340
min         8.240000
25%         9.280000
50%         9.360000
75%         9.440000
max        10.160000
Name: Afinnscore, dtype: float64
votazione 5 round Jones
count    904.000000
mean       9.351858
std        0.172628
min        8.320000
25%        9.280000
50%        9.360000
75%        9.360000
max       10.080000
Name: Afinnscore, dtype: float64


In [ ]:
#[MEDIA e STD AFINN SIA NORMALE CHE AGGIUSTATA]
mean_afinn_round5_tyson = tweets_5round_tyson['afinn'].mean()
mean_afinn_round5_jones = tweets_5round_jones['afinn'].mean()
mean_afinnscore_round5_tyson = tweets_5round_tyson['Afinnscore'].mean()
mean_afinnscore_round5_jones = tweets_5round_jones['Afinnscore'].mean()
std_afinn_round5_tyson = tweets_5round_tyson['afinn'].std()
std_afinn_round5_jones = tweets_5round_jones['afinn'].std()
std_afinnscore_round5_tyson = tweets_5round_tyson['Afinnscore'].std()
std_afinnscore_round5_jones = tweets_5round_jones['Afinnscore'].std()


### SENTIMENT ROUND 6 ##

In [ ]:
tweets_6round_tyson = datasetcompleto_incontro_tyson[datasetcompleto_incontro_tyson["ROUND"] == 6]
tweets_6round_tyson = tweets_6round_tyson.reset_index(drop = True)
print("Tweets presenti per il round 6 riferiti a Tyson:", len(tweets_6round_tyson))
tweets_6round_jones = datasetcompleto_incontro_jones[datasetcompleto_incontro_jones["ROUND"] == 6]
tweets_6round_jones = tweets_6round_jones.reset_index(drop = True)
print("Tweets presenti per il round 6 riferiti a Tyson:", len(tweets_6round_jones))

Tweets presenti per il round 6 riferiti a Tyson: 2773
Tweets presenti per il round 6 riferiti a Tyson: 802


In [ ]:
#AFINN SENTIMENT ROUND 6
tweets_6round_tyson['Text_clean_1']=tweets_6round_tyson['Text_clean_1'].apply(str)
tweets_6round_jones['Text_clean_1']=tweets_6round_jones['Text_clean_1'].apply(str)
tweets_6round_jones['afinn'] = tweets_6round_jones['Text_clean_1'].apply(afinn.score)
tweets_6round_tyson['afinn'] = tweets_6round_tyson['Text_clean_1'].apply(afinn.score)

In [ ]:
print("Describe di Afinn di Tyson del ROUND 6:")
print(tweets_6round_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones del ROUND 6:")
print(tweets_6round_jones['afinn'].describe())

Describe di Afinn di Tyson del ROUND 6:
count    2773.000000
mean        0.176704
std         2.438228
min       -17.000000
25%        -1.000000
50%         0.000000
75%         1.000000
max         9.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones del ROUND 6:
count    802.000000
mean      -0.218204
std        2.152377
min      -11.000000
25%       -1.000000
50%        0.000000
75%        0.000000
max        8.000000
Name: afinn, dtype: float64


In [ ]:
tweets_6round_jones['Afinnscore'] = rescale6(tweets_6round_jones['afinn'])
tweets_6round_tyson['Afinnscore'] = rescale6(tweets_6round_tyson['afinn'])
print('votazione 6 round Tyson ')
print(tweets_6round_tyson['Afinnscore'].describe())
print('votazione 6 round Jones')
print(tweets_6round_jones['Afinnscore'].describe())


votazione 6 round Tyson 
count    2773.000000
mean        9.017670
std         0.243823
min         7.300000
25%         8.900000
50%         9.000000
75%         9.100000
max         9.900000
Name: Afinnscore, dtype: float64
votazione 6 round Jones
count    802.000000
mean       8.978180
std        0.215238
min        7.900000
25%        8.900000
50%        9.000000
75%        9.000000
max        9.800000
Name: Afinnscore, dtype: float64


In [ ]:
#[MEDIA e STD AFINN SIA NORMALE CHE AGGIUSTATA]
mean_afinn_round6_tyson = tweets_6round_tyson['afinn'].mean()
mean_afinn_round6_jones = tweets_6round_jones['afinn'].mean()
mean_afinnscore_round6_tyson = tweets_6round_tyson['Afinnscore'].mean()
mean_afinnscore_round6_jones = tweets_6round_jones['Afinnscore'].mean()
std_afinn_round6_tyson = tweets_6round_tyson['afinn'].std()
std_afinn_round6_jones = tweets_6round_jones['afinn'].std()
std_afinnscore_round6_tyson = tweets_6round_tyson['Afinnscore'].std()
std_afinnscore_round6_jones = tweets_6round_jones['Afinnscore'].std()

### SENTIMENT ROUND 7 ###

In [ ]:
tweets_7round_tyson = datasetcompleto_incontro_tyson[datasetcompleto_incontro_tyson["ROUND"] == 7]
tweets_7round_tyson = tweets_7round_tyson.reset_index(drop = True)
print("Tweets presenti per il round 7 riferiti a Tyson:", len(tweets_7round_tyson))
tweets_7round_jones = datasetcompleto_incontro_jones[datasetcompleto_incontro_jones["ROUND"] == 7]
tweets_7round_jones = tweets_7round_jones.reset_index(drop = True)
print("Tweets presenti per il round 7 riferiti a Tyson:", len(tweets_7round_jones))

Tweets presenti per il round 7 riferiti a Tyson: 3474
Tweets presenti per il round 7 riferiti a Tyson: 1097


In [ ]:
#AFINN SENTIMENT ROUND 7
tweets_7round_tyson['Text_clean_1']=tweets_7round_tyson['Text_clean_1'].apply(str)
tweets_7round_jones['Text_clean_1']=tweets_7round_jones['Text_clean_1'].apply(str)
tweets_7round_jones['afinn'] = tweets_7round_jones['Text_clean_1'].apply(afinn.score)
tweets_7round_tyson['afinn'] = tweets_7round_tyson['Text_clean_1'].apply(afinn.score)

In [ ]:
print("Describe di Afinn di Tyson del ROUND 7:")
print(tweets_7round_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones del ROUND 7:")
print(tweets_7round_jones['afinn'].describe())

Describe di Afinn di Tyson del ROUND 7:
count    3474.000000
mean        0.161773
std         2.377617
min       -17.000000
25%        -1.000000
50%         0.000000
75%         1.000000
max        10.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones del ROUND 7:
count    1097.000000
mean       -0.142206
std         2.046409
min       -10.000000
25%        -1.000000
50%         0.000000
75%         0.000000
max         9.000000
Name: afinn, dtype: float64


In [ ]:
#AFINN SENTIMENT RESCALE ROUND 7
tweets_7round_jones['Afinnscore'] = rescale7(tweets_7round_jones['afinn'])
tweets_7round_tyson['Afinnscore'] = rescale7(tweets_7round_tyson['afinn'])
print('votazione 7 round Tyson')
print(tweets_7round_tyson['Afinnscore'].describe())
print('votazione 7 round Jones')
print(tweets_7round_jones['Afinnscore'].describe())


votazione 7 round Tyson
count    3474.000000
mean        9.013481
std         0.198135
min         7.583333
25%         8.916667
50%         9.000000
75%         9.083333
max         9.833333
Name: Afinnscore, dtype: float64
votazione 7 round Jones
count    1097.000000
mean        8.988149
std         0.170534
min         8.166667
25%         8.916667
50%         9.000000
75%         9.000000
max         9.750000
Name: Afinnscore, dtype: float64


In [ ]:
#[MEDIA e STD AFINN SIA NORMALE CHE AGGIUSTATA]
mean_afinn_round7_tyson = tweets_7round_tyson['afinn'].mean()
mean_afinn_round7_jones = tweets_7round_jones['afinn'].mean()
mean_afinnscore_round7_tyson = tweets_7round_tyson['Afinnscore'].mean()
mean_afinnscore_round7_jones = tweets_7round_jones['Afinnscore'].mean()
std_afinn_round7_tyson = tweets_7round_tyson['afinn'].std()
std_afinn_round7_jones = tweets_7round_jones['afinn'].std()
std_afinnscore_round7_tyson = tweets_7round_tyson['Afinnscore'].std()
std_afinnscore_round7_jones = tweets_7round_jones['Afinnscore'].std()


### SENTIMENT ROUND 8 ###

In [ ]:
tweets_8round_tyson = datasetcompleto_incontro_tyson[datasetcompleto_incontro_tyson["ROUND"] == 8]
tweets_8round_tyson = tweets_8round_tyson.reset_index(drop = True)
print("Tweets presenti per il round 8 riferiti a Tyson:", len(tweets_8round_tyson))
tweets_8round_jones = datasetcompleto_incontro_jones[datasetcompleto_incontro_jones["ROUND"] == 8]
tweets_8round_jones = tweets_8round_jones.reset_index(drop = True)
print("Tweets presenti per il round 8 riferiti a Tyson:", len(tweets_8round_jones))

Tweets presenti per il round 8 riferiti a Tyson: 9205
Tweets presenti per il round 8 riferiti a Tyson: 4067


In [ ]:
#AFINN SENTIMENT ROUND 8
tweets_8round_tyson['Text_clean_1']=tweets_8round_tyson['Text_clean_1'].apply(str)
tweets_8round_jones['Text_clean_1']=tweets_8round_jones['Text_clean_1'].apply(str)
tweets_8round_jones['afinn'] = tweets_8round_jones['Text_clean_1'].apply(afinn.score)
tweets_8round_tyson['afinn'] = tweets_8round_tyson['Text_clean_1'].apply(afinn.score)

In [ ]:
print("Describe di Afinn di Tyson del ROUND 8:")
print(tweets_8round_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones del ROUND 8:")
print(tweets_8round_jones['afinn'].describe())

Describe di Afinn di Tyson del ROUND 8:
count    9205.000000
mean        0.167735
std         2.502433
min       -14.000000
25%        -1.000000
50%         0.000000
75%         1.000000
max        12.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones del ROUND 8:
count    4067.000000
mean       -0.045242
std         2.034725
min       -12.000000
25%        -1.000000
50%         0.000000
75%         0.000000
max         9.000000
Name: afinn, dtype: float64


In [ ]:
#AFINN SENTIMENT ROUND 8 RESCALE
tweets_8round_jones['Afinnscore'] = rescale8(tweets_8round_jones['afinn'])
tweets_8round_tyson['Afinnscore'] = rescale8(tweets_8round_tyson['afinn'])
print('votazione 8 round Tyson ')
print(tweets_8round_tyson['Afinnscore'].describe())
print('votazione 8 round Jones')
print(tweets_8round_jones['Afinnscore'].describe())

votazione 8 round Tyson 
count    9205.000000
mean        9.089826
std         0.192495
min         8.000000
25%         9.000000
50%         9.076923
75%         9.153846
max        10.000000
Name: Afinnscore, dtype: float64
votazione 8 round Jones
count    4067.000000
mean        9.073443
std         0.156517
min         8.153846
25%         9.000000
50%         9.076923
75%         9.076923
max         9.769231
Name: Afinnscore, dtype: float64


In [ ]:
#[MEDIA e STD AFINN SIA NORMALE CHE AGGIUSTATA]
mean_afinn_round8_tyson = tweets_8round_tyson['afinn'].mean()
mean_afinn_round8_jones = tweets_8round_jones['afinn'].mean()
mean_afinnscore_round8_tyson = tweets_8round_tyson['Afinnscore'].mean()
mean_afinnscore_round8_jones = tweets_8round_jones['Afinnscore'].mean()
std_afinn_round8_tyson = tweets_8round_tyson['afinn'].std()
std_afinn_round8_jones = tweets_8round_jones['afinn'].std()
std_afinnscore_round8_tyson = tweets_8round_tyson['Afinnscore'].std()
std_afinnscore_round8_jones = tweets_8round_jones['Afinnscore'].std()

# SENTIMENT POST-INCONTRO #

In [ ]:
tweets_afterincontro_tyson = (dataset_tyson_def['Date_time_italy'] > "2020-11-29 06:23:00")
tweets_afterincontro_tyson = dataset_tyson_def.loc[tweets_afterincontro_tyson]
tweets_afterincontro_tyson = tweets_afterincontro_tyson.reset_index(drop = True)
print(len(tweets_afterincontro_tyson))
tweets_afterincontro_jones = (dataset_jones_def['Date_time_italy'] > "2020-11-29 06:23:00")
tweets_afterincontro_jones = dataset_jones_def.loc[tweets_afterincontro_jones]
tweets_afterincontro_jones = tweets_afterincontro_jones.reset_index(drop = True)
print(len(tweets_afterincontro_jones))

74928
50505


In [ ]:
#AFINN
tweets_afterincontro_tyson['Text_clean_1']=tweets_afterincontro_tyson['Text_clean_1'].apply(str)
tweets_afterincontro_jones['Text_clean_1']=tweets_afterincontro_jones['Text_clean_1'].apply(str)
tweets_afterincontro_jones['afinn'] = tweets_afterincontro_jones['Text_clean_1'].apply(afinn.score)
tweets_afterincontro_tyson['afinn'] = tweets_afterincontro_tyson['Text_clean_1'].apply(afinn.score)

In [ ]:
print("Describe di Afinn di Tyson postmatch:")
print(tweets_afterincontro_tyson['afinn'].describe())
print("\\\\\\\\\\\\\\")
print("Describe di Afinn di Jones postmatch:")
print(tweets_afterincontro_jones['afinn'].describe())

Describe di Afinn di Tyson postmatch:
count    74928.000000
mean         0.491138
std          2.478409
min        -20.000000
25%         -1.000000
50%          0.000000
75%          2.000000
max         20.000000
Name: afinn, dtype: float64
\\\\\\\
Describe di Afinn di Jones postmatch:
count    50505.000000
mean         0.369548
std          2.412482
min        -24.000000
25%          0.000000
50%          0.000000
75%          2.000000
max         20.000000
Name: afinn, dtype: float64


In [ ]:
tweets_afterincontro_jones['Afinnscore'] = rescaleafter(tweets_afterincontro_jones['afinn'])
tweets_afterincontro_tyson['Afinnscore'] = rescaleafter(tweets_afterincontro_tyson['afinn'])
print('votazione postfight Tyson')
print(tweets_afterincontro_tyson['Afinnscore'].describe())
print('votazione postfight Jones')
print(tweets_afterincontro_jones['Afinnscore'].describe())

votazione postfight Tyson
count    74928.000000
mean         9.113234
std          0.112655
min          8.181818
25%          9.045455
50%          9.090909
75%          9.181818
max         10.000000
Name: Afinnscore, dtype: float64
votazione postfight Jones
count    50505.000000
mean         9.107707
std          0.109658
min          8.000000
25%          9.090909
50%          9.090909
75%          9.181818
max         10.000000
Name: Afinnscore, dtype: float64


### INTEGRAZIONE DATASETS INCONTRI ###

In [ ]:
pdList_round_totali_tyson = [tweets_1round_tyson, tweets_2round_tyson, tweets_3round_tyson, tweets_4round_tyson, tweets_5round_tyson, tweets_6round_tyson, tweets_7round_tyson, tweets_8round_tyson]  
rounds_tyson_sentiment_completo = pd.concat(pdList_round_totali_tyson)
len(rounds_tyson_sentiment_completo)

28942

In [ ]:
pdList_round_totali_jones = [tweets_1round_jones, tweets_2round_jones, tweets_3round_jones, tweets_4round_jones, tweets_5round_jones, tweets_6round_jones, tweets_7round_jones, tweets_8round_jones]  
rounds_jones_sentiment_completo = pd.concat(pdList_round_totali_jones)
len(rounds_jones_sentiment_completo)

9850

In [ ]:
rounds_jones_sentiment_completo.columns

Index(['ROUND', '_id', 'Screen_name', 'Name', 'Text', 'Followers',
       'Number_Retweet', 'Position', 'Date_time_italy', 'Hashtags', 'hashtags',
       'Text_clean_1', 'Date_time_losangeles', 'Match_parola_Tyson',
       'Match_parola_Jones', 'Match_parola', 'TYSON_jab_landed',
       'Jab Landed / Thrown Tyson', 'JONES_jab_landed',
       'Jab Landed / Thrown Jones', 'pugni_tot_tyson', 'pugni_tot_jones',
       'jab_landed_tyson', 'jab_landed_jones', 'CUMSUM_pugni_tyson',
       'CUMSUM_pugni_jones', 'CUMSUM_jab_landed_tyson',
       'CUMSUM_jab_landed_jones', 'TYSON_punches_total',
       'Total Punches Landed / Thrown Tyson', 'JONES_punches_total',
       'Total Punches Landed / Thrown Jones', 'pugni_tirati_complex_tyson',
       'pugni_tirati_complex_jones', 'tot_punch_landed_tyson',
       'tot_punch_landed_jones', 'CUMSUM_pugni_tirati_tyson',
       'CUMSUM_pugni_tirati_jones', 'CUMSUM_pugni_landed_tyson',
       'CUMSUM_pugni_landed_jones', 'TYSON_power_total',
       'Power P

In [ ]:
tweets_incontro_tyson_test1 = (rounds_tyson_sentiment_completo.groupby([pd.Grouper(key='Date_time_italy', freq='1min')]).agg({'Afinnscore':'mean', 'afinn':'mean'}))
tweets_incontro_jones_test1 = (rounds_jones_sentiment_completo.groupby([pd.Grouper(key='Date_time_italy', freq='1min')]).agg({'Afinnscore':'mean', 'afinn':'mean'}))
tweets_incontro_tyson_test2 = (rounds_tyson_sentiment_completo.groupby([pd.Grouper(key='Date_time_italy', freq='1min')]).agg({'Afinnscore':'std', 'afinn':'std'}))
tweets_incontro_jones_test2 = (rounds_jones_sentiment_completo.groupby([pd.Grouper(key='Date_time_italy', freq='1min')]).agg({'Afinnscore':'std', 'afinn':'std'}))

In [ ]:
tweets_incontro_tyson_test1 = tweets_incontro_tyson_test1.reset_index()
tweets_incontro_jones_test1 = tweets_incontro_jones_test1.reset_index()
tweets_incontro_tyson_test2 = tweets_incontro_tyson_test2.reset_index()
tweets_incontro_jones_test2 = tweets_incontro_jones_test2.reset_index()

In [ ]:
tweets_incontro_tyson_test1 = tweets_incontro_tyson_test1.rename(columns={"Afinnscore": "Afinnscore_mean_tyson", "afinn": "afinn_mean_tyson"})
tweets_incontro_jones_test1 = tweets_incontro_jones_test1.rename(columns={"Afinnscore": "Afinnscore_mean_jones", "afinn": "afinn_mean_jones"})
tweets_incontro_tyson_test2 = tweets_incontro_tyson_test2.rename(columns={"Afinnscore": "Afinnscore_std_tyson", "afinn": "afinn_std_tyson"})
tweets_incontro_jones_test2 = tweets_incontro_jones_test2.rename(columns={"Afinnscore": "Afinnscore_std_jones", "afinn": "afinn_std_jones"})

In [ ]:
tweets_incontro_andamentisentiment = pd.merge(tweets_incontro_tyson_test1, tweets_incontro_jones_test1, how='left')
tweets_incontro_andamentisentiment = pd.merge(tweets_incontro_andamentisentiment, tweets_incontro_tyson_test2, how='left')
tweets_incontro_andamentisentiment = pd.merge(tweets_incontro_andamentisentiment, tweets_incontro_jones_test2, how='left')

In [ ]:
tweets_incontro_andamentisentiment['Afinnscore_mean_tyson'] = tweets_incontro_andamentisentiment['Afinnscore_mean_tyson'].astype(str)
tweets_incontro_andamentisentiment['Afinnscore_mean_jones'] = tweets_incontro_andamentisentiment['Afinnscore_mean_jones'].astype(str)
tweets_incontro_andamentisentiment['afinn_mean_tyson'] = tweets_incontro_andamentisentiment['afinn_mean_tyson'].astype(str)
tweets_incontro_andamentisentiment['afinn_mean_jones'] = tweets_incontro_andamentisentiment['afinn_mean_jones'].astype(str)
tweets_incontro_andamentisentiment['Afinnscore_std_tyson'] = tweets_incontro_andamentisentiment['Afinnscore_std_tyson'].astype(str)
tweets_incontro_andamentisentiment['Afinnscore_std_jones'] = tweets_incontro_andamentisentiment['Afinnscore_std_jones'].astype(str)
tweets_incontro_andamentisentiment['afinn_std_tyson'] = tweets_incontro_andamentisentiment['afinn_std_tyson'].astype(str)
tweets_incontro_andamentisentiment['afinn_std_jones'] = tweets_incontro_andamentisentiment['afinn_std_jones'].astype(str)

In [ ]:
tweets_incontro_andamentisentiment['Afinnscore_mean_tyson'] = tweets_incontro_andamentisentiment['Afinnscore_mean_tyson'].apply(lambda x: x.replace('.', ','))
tweets_incontro_andamentisentiment['Afinnscore_mean_jones'] = tweets_incontro_andamentisentiment['Afinnscore_mean_jones'].apply(lambda x: x.replace('.', ','))
tweets_incontro_andamentisentiment['afinn_mean_tyson'] = tweets_incontro_andamentisentiment['afinn_mean_tyson'].apply(lambda x: x.replace('.', ','))
tweets_incontro_andamentisentiment['afinn_mean_jones'] = tweets_incontro_andamentisentiment['afinn_mean_jones'].apply(lambda x: x.replace('.', ','))
tweets_incontro_andamentisentiment['Afinnscore_std_tyson'] = tweets_incontro_andamentisentiment['Afinnscore_std_tyson'].apply(lambda x: x.replace('.', ','))
tweets_incontro_andamentisentiment['Afinnscore_std_jones'] = tweets_incontro_andamentisentiment['Afinnscore_std_jones'].apply(lambda x: x.replace('.', ','))
tweets_incontro_andamentisentiment['afinn_std_tyson'] = tweets_incontro_andamentisentiment['afinn_std_tyson'].apply(lambda x: x.replace('.', ','))
tweets_incontro_andamentisentiment['afinn_std_jones'] = tweets_incontro_andamentisentiment['afinn_std_jones'].apply(lambda x: x.replace('.', ','))

In [ ]:
tweets_incontro_andamentisentiment.to_csv(r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/ESPERIMENTITABLEAU/tweets_incontro_andamentisentiment_def.csv')

# ANALISI AFTER MATCH #

In [ ]:
tweets_afterincontro_tyson_test1 = (tweets_afterincontro_tyson.groupby([pd.Grouper(key='Date_time_italy', freq='10min')]).agg({'Afinnscore':'mean', 'afinn':'mean'}))
tweets_afterincontro_jones_test1 = (tweets_afterincontro_jones.groupby([pd.Grouper(key='Date_time_italy', freq='10min')]).agg({'Afinnscore':'mean', 'afinn':'mean'}))
tweets_afterincontro_tyson_test2 = (tweets_afterincontro_tyson.groupby([pd.Grouper(key='Date_time_italy', freq='10min')]).agg({'Afinnscore':'std', 'afinn':'std'}))
tweets_afterincontro_jones_test2 = (tweets_afterincontro_jones.groupby([pd.Grouper(key='Date_time_italy', freq='10min')]).agg({'Afinnscore':'std', 'afinn':'std'}))

In [ ]:
tweets_afterincontro_tyson_test1 = tweets_afterincontro_tyson_test1.reset_index()
tweets_afterincontro_jones_test1 = tweets_afterincontro_jones_test1.reset_index()
tweets_afterincontro_tyson_test2 = tweets_afterincontro_tyson_test2.reset_index()
tweets_afterincontro_jones_test2 = tweets_afterincontro_jones_test2.reset_index()

In [ ]:
tweets_afterincontro_tyson_test1 = tweets_afterincontro_tyson_test1.rename(columns={"Afinnscore": "Afinnscore_mean_tyson", "afinn": "afinn_mean_tyson"})
tweets_afterincontro_jones_test1 = tweets_afterincontro_jones_test1.rename(columns={"Afinnscore": "Afinnscore_mean_jones", "afinn": "afinn_mean_jones"})
tweets_afterincontro_tyson_test2 = tweets_afterincontro_tyson_test2.rename(columns={"Afinnscore": "Afinnscore_std_tyson", "afinn": "afinn_std_tyson"})
tweets_afterincontro_jones_test2 = tweets_afterincontro_jones_test2.rename(columns={"Afinnscore": "Afinnscore_std_jones", "afinn": "afinn_std_jones"})

In [ ]:
tweets_afterincontro_andamentisentiment = pd.merge(tweets_afterincontro_tyson_test1, tweets_afterincontro_jones_test1, how='left')
tweets_afterincontro_andamentisentiment = pd.merge(tweets_afterincontro_andamentisentiment, tweets_afterincontro_tyson_test2, how='left')
tweets_afterincontro_andamentisentiment = pd.merge(tweets_afterincontro_andamentisentiment, tweets_afterincontro_jones_test2, how='left')

In [ ]:
tweets_afterincontro_andamentisentiment.head()

,Date_time_italy,Afinnscore_mean_tyson,afinn_mean_tyson,Afinnscore_mean_jones,afinn_mean_jones,Afinnscore_std_tyson,afinn_std_tyson,Afinnscore_std_jones,afinn_std_jones
0,2020-11-29 06:20:00,9.103623,0.279711,9.098122,0.158686,0.118513,2.607283,0.087790,1.931382
1,2020-11-29 06:30:00,9.109310,0.404815,9.085540,-0.118118,0.125799,2.767571,0.099696,2.193301
2,2020-11-29 06:40:00,9.120674,0.654824,9.090852,-0.001254,0.119694,2.633275,0.094107,2.070355
3,2020-11-29 06:50:00,9.117392,0.582627,9.090535,-0.008236,0.111336,2.449394,0.087000,1.913999
4,2020-11-29 07:00:00,9.103815,0.283932,9.093563,0.058394,0.103787,2.283321,0.086965,1.913235


In [ ]:
tweets_afterincontro_andamentisentiment['Afinnscore_mean_tyson'] = tweets_afterincontro_andamentisentiment['Afinnscore_mean_tyson'].astype(str)
tweets_afterincontro_andamentisentiment['Afinnscore_mean_jones'] = tweets_afterincontro_andamentisentiment['Afinnscore_mean_jones'].astype(str)
tweets_afterincontro_andamentisentiment['afinn_mean_tyson'] = tweets_afterincontro_andamentisentiment['afinn_mean_tyson'].astype(str)
tweets_afterincontro_andamentisentiment['afinn_mean_jones'] = tweets_afterincontro_andamentisentiment['afinn_mean_jones'].astype(str)
tweets_afterincontro_andamentisentiment['Afinnscore_std_tyson'] = tweets_afterincontro_andamentisentiment['Afinnscore_std_tyson'].astype(str)
tweets_afterincontro_andamentisentiment['Afinnscore_std_jones'] = tweets_afterincontro_andamentisentiment['Afinnscore_std_jones'].astype(str)
tweets_afterincontro_andamentisentiment['afinn_std_tyson'] = tweets_afterincontro_andamentisentiment['afinn_std_tyson'].astype(str)
tweets_afterincontro_andamentisentiment['afinn_std_jones'] = tweets_afterincontro_andamentisentiment['afinn_std_jones'].astype(str)

In [ ]:
tweets_afterincontro_andamentisentiment['Afinnscore_mean_tyson'] = tweets_afterincontro_andamentisentiment['Afinnscore_mean_tyson'].apply(lambda x: x.replace('.', ','))
tweets_afterincontro_andamentisentiment['Afinnscore_mean_jones'] = tweets_afterincontro_andamentisentiment['Afinnscore_mean_jones'].apply(lambda x: x.replace('.', ','))
tweets_afterincontro_andamentisentiment['afinn_mean_tyson'] = tweets_afterincontro_andamentisentiment['afinn_mean_tyson'].apply(lambda x: x.replace('.', ','))
tweets_afterincontro_andamentisentiment['afinn_mean_jones'] = tweets_afterincontro_andamentisentiment['afinn_mean_jones'].apply(lambda x: x.replace('.', ','))
tweets_afterincontro_andamentisentiment['Afinnscore_std_tyson'] = tweets_afterincontro_andamentisentiment['Afinnscore_std_tyson'].apply(lambda x: x.replace('.', ','))
tweets_afterincontro_andamentisentiment['Afinnscore_std_jones'] = tweets_afterincontro_andamentisentiment['Afinnscore_std_jones'].apply(lambda x: x.replace('.', ','))
tweets_afterincontro_andamentisentiment['afinn_std_tyson'] = tweets_afterincontro_andamentisentiment['afinn_std_tyson'].apply(lambda x: x.replace('.', ','))
tweets_afterincontro_andamentisentiment['afinn_std_jones'] = tweets_afterincontro_andamentisentiment['afinn_std_jones'].apply(lambda x: x.replace('.', ','))

In [ ]:
tweets_afterincontro_andamentisentiment.to_csv(r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/ESPERIMENTITABLEAU/tweets_afterincontro_andamentisentiment.csv')

### INTEGRAZIONE COMPLESSIVO ###

#### Statistiche match: ###

In [ ]:
statistichematch_completo = pd.merge(incontro_jab_landed, incontro_punches_total, how='left')
statistichematch_completo = pd.merge(statistichematch_completo, incontro_powerpunches_landed, how='left')
statistichematch_completo = pd.merge(statistichematch_completo, tabellino_incontro_t, how='left')
statistichematch_completo


,TYSON_jab_landed,Jab Landed / Thrown Tyson,JONES_jab_landed,Jab Landed / Thrown Jones,ROUND,pugni_tot_tyson,pugni_tot_jones,jab_landed_tyson,jab_landed_jones,CUMSUM_pugni_tyson,...,tot_punch_tirati_tyson,tot_punch_tirati_jones,CUMSUM_power_tirati_tyson,CUMSUM_power_tirati_jones,CUMSUM_pugni_2_landed_tyson,CUMSUM_pugni_2_landed_jones,TYSON_punteggio_sing,JONES_punteggio_sing,CUMSUM_punteggio_tyson,CUMSUM_punteggio_jones
0,1/11,0.091,2/16,0.125,1,11.0,16.0,1.0,2.0,11.0,...,6.0,0.0,11.0,2.0,6.0,0.0,10.0,9.0,10.0,9.0
1,0/5,0.000,2/13,0.154,2,5.0,13.0,0.0,2.0,16.0,...,8.0,2.0,27.0,21.0,14.0,2.0,10.0,9.0,20.0,18.0
2,1/7,0.143,0/6,0.000,3,7.0,6.0,1.0,0.0,23.0,...,8.0,6.0,48.0,45.0,22.0,8.0,9.0,10.0,29.0,28.0
3,2/6,0.333,0/12,0.000,4,6.0,12.0,2.0,0.0,29.0,...,4.0,3.0,63.0,61.0,26.0,11.0,10.0,9.0,39.0,37.0
4,3/10,0.300,3/13,0.231,5,10.0,13.0,3.0,3.0,39.0,...,9.0,6.0,83.0,90.0,35.0,17.0,10.0,9.0,49.0,46.0
5,1/7,0.143,1/10,0.100,6,7.0,10.0,1.0,1.0,46.0,...,5.0,5.0,95.0,108.0,40.0,22.0,9.0,10.0,58.0,56.0
6,2/6,0.333,0/6,0.000,7,6.0,6.0,2.0,0.0,52.0,...,6.0,2.0,112.0,127.0,46.0,24.0,9.0,10.0,67.0,66.0
7,0/4,0.000,1/9,0.111,8,4.0,9.0,0.0,1.0,56.0,...,11.0,4.0,137.0,151.0,57.0,28.0,9.0,10.0,76.0,76.0


In [ ]:
#Creazione delle nuove liste:
#[MEAN:]
mean_afinnscore_tyson = [mean_afinnscore_round1_tyson, mean_afinnscore_round2_tyson, mean_afinnscore_round3_tyson, mean_afinnscore_round4_tyson, mean_afinnscore_round5_tyson, mean_afinnscore_round6_tyson, mean_afinnscore_round7_tyson, mean_afinnscore_round8_tyson] 
mean_afinnscore_jones = [mean_afinnscore_round1_jones, mean_afinnscore_round2_jones, mean_afinnscore_round3_jones, mean_afinnscore_round4_jones, mean_afinnscore_round5_jones, mean_afinnscore_round6_jones, mean_afinnscore_round7_jones, mean_afinnscore_round8_jones] 
mean_afinn_tyson = [mean_afinn_round1_tyson, mean_afinn_round2_tyson, mean_afinn_round3_tyson, mean_afinn_round4_tyson, mean_afinn_round5_tyson, mean_afinn_round6_tyson, mean_afinn_round7_tyson, mean_afinn_round8_tyson]
mean_afinn_jones = [mean_afinn_round1_jones, mean_afinn_round2_jones, mean_afinn_round3_jones, mean_afinn_round4_jones, mean_afinn_round5_jones, mean_afinn_round6_jones, mean_afinn_round7_jones, mean_afinn_round8_jones] 
#mean_afinnnorm_tyson = [mean_afinnnorm_round1_tyson, mean_afinnnorm_round2_tyson, mean_afinnnorm_round3_tyson, mean_afinnnorm_round4_tyson, mean_afinnnorm_round5_tyson, mean_afinnnorm_round6_tyson, mean_afinnnorm_round7_tyson, mean_afinnnorm_round8_tyson]
#mean_afinnnorm_jones = [mean_afinnnorm_round1_jones, mean_afinnnorm_round2_jones, mean_afinnnorm_round3_jones, mean_afinnnorm_round4_jones, mean_afinnnorm_round5_jones, mean_afinnnorm_round6_jones, mean_afinnnorm_round7_jones, mean_afinnnorm_round8_jones] 
#[STD:]
std_afinnscore_tyson = [std_afinnscore_round1_tyson, std_afinnscore_round2_tyson, std_afinnscore_round3_tyson, std_afinnscore_round4_tyson, std_afinnscore_round5_tyson, std_afinnscore_round6_tyson, std_afinnscore_round7_tyson, std_afinnscore_round8_tyson] 
std_afinnscore_jones = [std_afinnscore_round1_jones, std_afinnscore_round2_jones, std_afinnscore_round3_jones, std_afinnscore_round4_jones, std_afinnscore_round5_jones, std_afinnscore_round6_jones, std_afinnscore_round7_jones, std_afinnscore_round8_jones] 
std_afinn_tyson = [std_afinn_round1_tyson, std_afinn_round2_tyson, std_afinn_round3_tyson, std_afinn_round4_tyson, std_afinn_round5_tyson, std_afinn_round6_tyson, std_afinn_round7_tyson, std_afinn_round8_tyson]
std_afinn_jones = [std_afinn_round1_jones, std_afinn_round2_jones, std_afinn_round3_jones, std_afinn_round4_jones, std_afinn_round5_jones, std_afinn_round6_jones, std_afinn_round7_jones, std_afinn_round8_jones] 
#Vado a creare le nuove colonne:
statistichematch_completo['mean_afinnscore_tyson'] = mean_afinnscore_tyson
statistichematch_completo['mean_afinnscore_jones'] = mean_afinnscore_jones
statistichematch_completo['mean_afinn_tyson'] = mean_afinn_tyson
statistichematch_completo['mean_afinn_jones'] = mean_afinn_jones
statistichematch_completo['std_afinnscore_tyson'] = std_afinnscore_tyson
statistichematch_completo['std_afinnscore_jones'] = std_afinnscore_jones
statistichematch_completo['std_afinn_tyson'] = std_afinn_tyson
statistichematch_completo['std_afinn_jones'] = std_afinn_jones
#statistichematch_completo['mean_afinnnorm_tyson'] = mean_afinnnorm_tyson
#statistichematch_completo['mean_afinnnorm_jones'] = mean_afinnnorm_jones

In [ ]:
statistichematch_completo.dtypes

TYSON_jab_landed                        object
Jab Landed / Thrown Tyson              float64
JONES_jab_landed                        object
Jab Landed / Thrown Jones              float64
ROUND                                   object
pugni_tot_tyson                        float64
pugni_tot_jones                        float64
jab_landed_tyson                       float64
jab_landed_jones                       float64
CUMSUM_pugni_tyson                     float64
CUMSUM_pugni_jones                     float64
CUMSUM_jab_landed_tyson                float64
CUMSUM_jab_landed_jones                float64
TYSON_punches_total                     object
Total Punches Landed / Thrown Tyson    float64
JONES_punches_total                     object
Total Punches Landed / Thrown Jones    float64
pugni_tirati_complex_tyson             float64
pugni_tirati_complex_jones             float64
tot_punch_landed_tyson                 float64
tot_punch_landed_jones                 float64
CUMSUM_pugni_

In [ ]:
statistichematch_completo['pugni_tot_tyson'] = statistichematch_completo['pugni_tot_tyson'].astype(str)
statistichematch_completo['Jab Landed / Thrown Tyson'] = statistichematch_completo['Jab Landed / Thrown Tyson'].astype(str)
statistichematch_completo['Jab Landed / Thrown Jones'] = statistichematch_completo['Jab Landed / Thrown Jones'].astype(str)
statistichematch_completo['pugni_tot_jones'] = statistichematch_completo['pugni_tot_jones'].astype(str)
statistichematch_completo['jab_landed_tyson'] = statistichematch_completo['jab_landed_tyson'].astype(str)
statistichematch_completo['Total Punches Landed / Thrown Tyson'] = statistichematch_completo['Total Punches Landed / Thrown Tyson'].astype(str)
statistichematch_completo['Total Punches Landed / Thrown Jones'] = statistichematch_completo['Total Punches Landed / Thrown Jones'].astype(str)
statistichematch_completo['jab_landed_jones'] = statistichematch_completo['jab_landed_jones'].astype(str)
statistichematch_completo['CUMSUM_pugni_tyson'] = statistichematch_completo['CUMSUM_pugni_tyson'].astype(str)
statistichematch_completo['CUMSUM_pugni_jones'] = statistichematch_completo['CUMSUM_pugni_jones'].astype(str)
statistichematch_completo['CUMSUM_jab_landed_tyson'] = statistichematch_completo['CUMSUM_jab_landed_tyson'].astype(str)
statistichematch_completo['CUMSUM_jab_landed_jones'] = statistichematch_completo['CUMSUM_jab_landed_jones'].astype(str)
statistichematch_completo['TYSON_punches_total'] = statistichematch_completo['TYSON_punches_total'].astype(str)
statistichematch_completo['JONES_punches_total'] = statistichematch_completo['JONES_punches_total'].astype(str)
statistichematch_completo['pugni_tirati_complex_tyson'] = statistichematch_completo['pugni_tirati_complex_tyson'].astype(str)
statistichematch_completo['pugni_tirati_complex_jones'] = statistichematch_completo['pugni_tirati_complex_jones'].astype(str)
statistichematch_completo['tot_punch_landed_tyson'] = statistichematch_completo['tot_punch_landed_tyson'].astype(str)
statistichematch_completo['tot_punch_landed_jones'] = statistichematch_completo['tot_punch_landed_jones'].astype(str)
statistichematch_completo['CUMSUM_pugni_tirati_tyson'] = statistichematch_completo['CUMSUM_pugni_tirati_tyson'].astype(str)
statistichematch_completo['CUMSUM_pugni_tirati_jones'] = statistichematch_completo['CUMSUM_pugni_tirati_jones'].astype(str)
statistichematch_completo['CUMSUM_pugni_landed_tyson'] = statistichematch_completo['CUMSUM_pugni_landed_tyson'].astype(str)
statistichematch_completo['CUMSUM_pugni_landed_jones'] = statistichematch_completo['CUMSUM_pugni_landed_jones'].astype(str)
statistichematch_completo['TYSON_power_total'] = statistichematch_completo['TYSON_power_total'].astype(str)
statistichematch_completo['JONES_power_total'] = statistichematch_completo['JONES_power_total'].astype(str)
statistichematch_completo['power_pugni_tirati_tyson'] = statistichematch_completo['power_pugni_tirati_tyson'].astype(str)
statistichematch_completo['power_pugni_tirati_jones'] = statistichematch_completo['power_pugni_tirati_jones'].astype(str)
statistichematch_completo['CUMSUM_power_tirati_tyson'] = statistichematch_completo['CUMSUM_power_tirati_tyson'].astype(str)
statistichematch_completo['CUMSUM_power_tirati_jones'] = statistichematch_completo['CUMSUM_power_tirati_jones'].astype(str)
statistichematch_completo['TYSON_punteggio_sing'] = statistichematch_completo['TYSON_punteggio_sing'].astype(str)
statistichematch_completo['JONES_punteggio_sing'] = statistichematch_completo['JONES_punteggio_sing'].astype(str)
statistichematch_completo['CUMSUM_punteggio_tyson'] = statistichematch_completo['CUMSUM_punteggio_tyson'].astype(str)
statistichematch_completo['CUMSUM_punteggio_jones'] = statistichematch_completo['CUMSUM_punteggio_jones'].astype(str)
statistichematch_completo['mean_afinnscore_tyson'] = statistichematch_completo['mean_afinnscore_tyson'].astype(str)
statistichematch_completo['mean_afinnscore_jones'] = statistichematch_completo['mean_afinnscore_jones'].astype(str)
statistichematch_completo['mean_afinn_tyson'] = statistichematch_completo['mean_afinn_tyson'].astype(str)
statistichematch_completo['mean_afinn_jones'] = statistichematch_completo['mean_afinn_jones'].astype(str)
statistichematch_completo['std_afinnscore_tyson'] = statistichematch_completo['std_afinn_tyson'].astype(str)
statistichematch_completo['std_afinnscore_jones'] = statistichematch_completo['std_afinn_jones'].astype(str)
statistichematch_completo['std_afinn_tyson'] = statistichematch_completo['std_afinn_tyson'].astype(str)
statistichematch_completo['std_afinn_jones'] = statistichematch_completo['std_afinn_jones'].astype(str)
statistichematch_completo['Power Punches landed / Thrown Tyson'] = statistichematch_completo['Power Punches landed / Thrown Tyson'].astype(str)
statistichematch_completo['Power Punches landed / Thrown Jones'] = statistichematch_completo['Power Punches landed / Thrown Jones'].astype(str)

#statistichematch_completo['mean_afinnnorm_jones'] = statistichematch_completo['mean_afinnnorm_jones'].apply(lambda x: x.replace('.', ','))

In [ ]:
statistichematch_completo['pugni_tot_tyson'] = statistichematch_completo['pugni_tot_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['Jab Landed / Thrown Tyson'] = statistichematch_completo['Jab Landed / Thrown Tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['Jab Landed / Thrown Jones'] = statistichematch_completo['Jab Landed / Thrown Jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['pugni_tot_jones'] = statistichematch_completo['pugni_tot_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['jab_landed_tyson'] = statistichematch_completo['jab_landed_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['jab_landed_jones'] = statistichematch_completo['jab_landed_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_pugni_tyson'] = statistichematch_completo['CUMSUM_pugni_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_pugni_jones'] = statistichematch_completo['CUMSUM_pugni_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_jab_landed_tyson'] = statistichematch_completo['CUMSUM_jab_landed_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_jab_landed_jones'] = statistichematch_completo['CUMSUM_jab_landed_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['TYSON_punches_total'] = statistichematch_completo['TYSON_punches_total'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['JONES_punches_total'] = statistichematch_completo['JONES_punches_total'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['pugni_tirati_complex_tyson'] = statistichematch_completo['pugni_tirati_complex_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['pugni_tirati_complex_jones'] = statistichematch_completo['pugni_tirati_complex_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['tot_punch_landed_tyson'] = statistichematch_completo['tot_punch_landed_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['tot_punch_landed_jones'] = statistichematch_completo['tot_punch_landed_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_pugni_tirati_tyson'] = statistichematch_completo['CUMSUM_pugni_tirati_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_pugni_tirati_jones'] = statistichematch_completo['CUMSUM_pugni_tirati_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_pugni_landed_tyson'] = statistichematch_completo['CUMSUM_pugni_landed_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_pugni_landed_jones'] = statistichematch_completo['CUMSUM_pugni_landed_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['TYSON_power_total'] = statistichematch_completo['TYSON_power_total'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['JONES_power_total'] = statistichematch_completo['JONES_power_total'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['power_pugni_tirati_tyson'] = statistichematch_completo['power_pugni_tirati_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['power_pugni_tirati_jones'] = statistichematch_completo['power_pugni_tirati_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_power_tirati_tyson'] = statistichematch_completo['CUMSUM_power_tirati_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_power_tirati_jones'] = statistichematch_completo['CUMSUM_power_tirati_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['TYSON_punteggio_sing'] = statistichematch_completo['TYSON_punteggio_sing'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['JONES_punteggio_sing'] = statistichematch_completo['JONES_punteggio_sing'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_punteggio_tyson'] = statistichematch_completo['CUMSUM_punteggio_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['CUMSUM_punteggio_jones'] = statistichematch_completo['CUMSUM_punteggio_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['mean_afinnscore_tyson'] = statistichematch_completo['mean_afinnscore_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['mean_afinnscore_jones'] = statistichematch_completo['mean_afinnscore_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['mean_afinn_tyson'] = statistichematch_completo['mean_afinn_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['mean_afinn_jones'] = statistichematch_completo['mean_afinn_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['std_afinnscore_tyson'] = statistichematch_completo['std_afinn_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['std_afinnscore_jones'] = statistichematch_completo['std_afinn_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['std_afinn_tyson'] = statistichematch_completo['std_afinn_tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['std_afinn_jones'] = statistichematch_completo['std_afinn_jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['Total Punches Landed / Thrown Tyson'] = statistichematch_completo['Total Punches Landed / Thrown Tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['Total Punches Landed / Thrown Jones'] = statistichematch_completo['Total Punches Landed / Thrown Jones'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['Power Punches landed / Thrown Tyson'] = statistichematch_completo['Power Punches landed / Thrown Tyson'].apply(lambda x: x.replace('.', ','))
statistichematch_completo['Power Punches landed / Thrown Jones'] = statistichematch_completo['Power Punches landed / Thrown Jones'].apply(lambda x: x.replace('.', ','))
#statistichematch_completo['pugni_tot_tyson'] = statistichematch_completo['pugni_tot_tyson'].apply(lambda x: x.replace('.', ','))
#statistichematch_completo['mean_afinnscore_tyson'] = statistichematch_completo['mean_afinnscore_tyson'].apply(lambda x: x.replace('.', ','))

In [ ]:
statistichematch_completo

,TYSON_jab_landed,Jab Landed / Thrown Tyson,JONES_jab_landed,Jab Landed / Thrown Jones,ROUND,pugni_tot_tyson,pugni_tot_jones,jab_landed_tyson,jab_landed_jones,CUMSUM_pugni_tyson,...,CUMSUM_punteggio_tyson,CUMSUM_punteggio_jones,mean_afinnscore_tyson,mean_afinnscore_jones,mean_afinn_tyson,mean_afinn_jones,std_afinnscore_tyson,std_afinnscore_jones,std_afinn_tyson,std_afinn_jones
0,1/11,"0,091",2/16,"0,125",1,"11,0","16,0","1,0","2,0","11,0",...,"10,0","9,0","9,04862664761846","9,036463842342657","0,05920644761314321","-0,08066581306017925","2,2793778267448976","2,530814719782594","2,2793778267448976","2,530814719782594"
1,0/5,"0,0",2/13,"0,154",2,"5,0","13,0","0,0","2,0","16,0",...,"20,0","18,0","9,176517660044235","9,185547785547802","0,11821192052980133","0,22657342657342658","2,5899388961991954","2,228946035018326","2,5899388961991954","2,228946035018326"
2,1/7,"0,143",0/6,"0,0",3,"7,0","6,0","1,0","0,0","23,0",...,"29,0","28,0","9,09552962920181","9,083838383838353","0,05082592121982211","-0,07777777777777778","2,5544826742061426","2,369485661305987","2,5544826742061426","2,369485661305987"
3,2/6,"0,333",0/12,"0,0",4,"6,0","12,0","2,0","0,0","29,0",...,"39,0","37,0","9,176597382602017","9,151457883369343","0,11916859122401847","-0,18250539956803455","2,583830577740358","2,1840098473872085","2,583830577740358","2,1840098473872085"
4,3/10,"0,3",3/13,"0,231",5,"10,0","13,0","3,0","3,0","39,0",...,"49,0","46,0","9,371162152896972","9,34168553459113","0,1395269112101474","-0,2289308176100629","2,4176571368277635","2,094232992697202","2,4176571368277635","2,094232992697202"
5,1/7,"0,143",1/10,"0,1",6,"7,0","10,0","1,0","1,0","46,0",...,"58,0","56,0","9,011081492764657","8,992926829268283","0,11081492764661081","-0,07073170731707316","2,3543882316142644","2,0766442243201904","2,3543882316142644","2,0766442243201904"
6,2/6,"0,333",0/6,"0,0",7,"6,0","6,0","2,0","0,0","52,0",...,"67,0","66,0","9,01420295924064","8,96809972516687","0,1704355108877722","-0,3828032979976443","2,3800624828596995","2,0742664446007586","2,3800624828596995","2,0742664446007586"
7,0/4,"0,0",1/9,"0,111",8,"4,0","9,0","0,0","1,0","56,0",...,"76,0","76,0","9,091371197636848","9,079866928342202","0,18782556928114302","0,03827006845052894","2,538801717483598","2,007943514095569","2,538801717483598","2,007943514095569"


In [ ]:
statistichematch_completo.to_csv(r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/ESPERIMENTITABLEAU/statistichematch_completo.csv')

# UNIONE ANDAMENTI SENTIMENT # 

In [ ]:
sentimentminuti_completo = pd.merge(tweets_preincontro_andamentisentiment, tweets_incontro_andamentisentiment, how='left')
sentimentminuti_completo = pd.merge(sentimentminuti_completo, tweets_afterincontro_andamentisentiment, how='left')
sentimentminuti_completo

,Date_time_italy,Afinnscore_mean_tyson,afinn_mean_tyson,Afinnscore_mean_jones,afinn_mean_jones,Afinnscore_std_tyson,afinn_std_tyson,Afinnscore_std_jones,afinn_std_jones
0,2020-11-27 14:30:00,"9,072346179851252","-2,6551724137931036","9,217927170868345","1,0571428571428572","0,11916659301202395","3,0387481218066186","0,09053661804130518","2,3086837600532792"
1,2020-11-27 14:40:00,"9,148459383753496","-0,7142857142857143","9,22598313068244","1,2625698324022345","0,12045955902175376","3,0717187550547176","0,09751608471209045","2,486660160158302"
2,2020-11-27 14:50:00,"9,126860382707298","-1,2650602409638554","9,196899224806195","0,5209302325581395","0,11892882773532926","3,032685107250901","0,11554085006966647","2,946291676776499"
3,2020-11-27 15:00:00,"9,140133117467162","-0,926605504587156","9,208287088420274","0,8113207547169812","0,11241151395868734","2,8664936059465376","0,10675915102079064","2,722358351030163"
4,2020-11-27 15:10:00,"9,152129817444216","-0,6206896551724138","9,19401916017549","0,4474885844748858","0,11366436700595046","2,898441358651733","0,10023256586783598","2,555930429629808"
...,...,...,...,...,...,...,...,...,...
232,2020-11-29 05:10:00,"9,182630341001289","0,157073695535234","9,187545711001928","0,2824156305506217","0,10804715408407296","2,7552024291438637","0,08314125580929509","2,1201020231370222"
233,2020-11-29 05:20:00,"9,193818990456114","0,4423842566260949","9,189506768098921","0,3324225865209472","0,1188091406255537","3,0296330859516187","0,09501412406599946","2,422860163682986"
234,2020-11-29 05:30:00,"9,184869749998967","0,21417862497322768","9,181875102982305","0,13781512605042018","0,11252848686182694","2,869476414976581","0,08933716551734407","2,278097720692275"
235,2020-11-29 05:40:00,"9,185058886678714","0,21900161030595813","9,188780143275242","0,313893653516295","0,10869075404331639","2,7716142281045677","0,08489028182194257","2,164702186459535"


In [ ]:
sentimentminuti_completo.to_csv(r'/Users/matteo/Desktop/DATA SCIENCE/Data Management/Progettto/ESPERIMENTITABLEAU/sentimentminuti_completo.csv')